# Instal library

In [ ]:
# Impor library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Untuk visualisasi interaktif
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Atur gaya tampilan
plt.style.use('ggplot')


# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD, FastICA, NMF
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, SpectralEmbedding
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, OPTICS, SpectralClustering, Birch
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.feature_selection import SelectKBest, mutual_info_regression, VarianceThreshold
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from scipy.stats import zscore, spearmanr, pearsonr
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform
from statsmodels.stats.outliers_influence import variance_inflation_factor
from kneed import KneeLocator
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.mixture import GaussianMixture
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Import library yang diperlukan
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from umap import UMAP  # Jika umap tidak terinstall, gunakan: pip install umap-learn

# Untuk visualisasi color-blind friendly dengan Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data Cleaning & Preparation

In [ ]:
# Baca file CSV
df = pd.read_csv('./data/employee_data.csv')

# Tampilkan 5 baris pertama untuk melihat struktur data
display(df.head())

In [ ]:
# Informasi tentang dataset
print("Ukuran dataset:", df.shape)
print("\nInformasi tentang tipe data:")
display(df.info())

In [ ]:
# Statistik deskriptif data numerik
display(df.describe())

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

# Feature Engineering

## Salary Category

In [ ]:
def salary_category(income):
    if income < 5000:
        return 'Rendah (< 5000)'
    elif income < 10000:
        return 'Sedang (5000-10000)'
    elif income < 15000:
        return 'Tinggi (10000-15000)'
    else:
        return 'Sangat Tinggi (>15000)'

df['SalaryCategory'] = df['MonthlyIncome'].apply(salary_category)

## Promotion Category

In [ ]:
def promotion_category(years):
    if years == 0:
        return 'Baru Dipromosikan'
    elif years <= 2:
        return '1-2 Tahun'
    elif years <= 5:
        return '3-5 Tahun'
    else:
        return '> 5 Tahun'

df['PromotionCategory'] = df['YearsSinceLastPromotion'].apply(promotion_category)

## Age Category

In [ ]:
# Buat kategori usia
def age_category(age):
    if age < 30:
        return '< 30'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    else:
        return '50+'

df['AgeGroup'] = df['Age'].apply(age_category)

## Satisfaction Category

In [ ]:
satisfaction_cols = ['EnvironmentSatisfaction', 'JobSatisfaction', 
                     'RelationshipSatisfaction', 'WorkLifeBalance']
satisfaction_labels = {1: 'Rendah', 2: 'Sedang', 3: 'Tinggi', 4: 'Sangat Tinggi'}

df_satisfaction = df.copy()
for col in satisfaction_cols:
    df_satisfaction[col] = df_satisfaction[col].map(satisfaction_labels)

# Exploratory Data Analysis (EDA)

## Analisis Univariate

### Distribusi Attrition

In [ ]:
attrition_counts = df['Attrition'].value_counts()
attrition_percentage = df['Attrition'].value_counts(normalize=True) * 100

attrition_df = pd.DataFrame({
    'Count': attrition_counts,
    'Percentage': attrition_percentage
})

print("Distribusi Attrition:")
display(attrition_df)

fig = px.pie(
    names=['Tidak Keluar', 'Keluar'] if 0 in attrition_counts.index else ['Keluar', 'Tidak Keluar'],
    values=attrition_counts.values,
    title='Persentase Karyawan yang Keluar vs Bertahan',
    color_discrete_sequence=px.colors.qualitative.Set3,
    hole=0.4
)
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

fig.write_image("./visualisasi/presentase_attrition.jpg", scale=1)

## Analisis Bivariate

### Attrition by Age

In [ ]:
age_group_attrition = df.groupby('AgeGroup')['Attrition'].mean() * 100
age_group_counts = df.groupby('AgeGroup').size()

order = ['< 30', '30-39', '40-49', '50+']
age_group_attrition = age_group_attrition.reindex(order)
age_group_counts = age_group_counts.reindex(order)

age_group_df = pd.DataFrame({
    'Jumlah Karyawan': age_group_counts,
    'Tingkat Attrition (%)': age_group_attrition
})

display(age_group_df)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        x=age_group_df.index,
        y=age_group_df['Jumlah Karyawan'],
        name='Jumlah Karyawan',
        marker_color='royalblue',
        opacity=0.7
    ),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=age_group_df.index,
        y=age_group_df['Tingkat Attrition (%)'],
        name='Tingkat Attrition (%)',
        marker_color='firebrick',
        mode='lines+markers',
        marker=dict(size=10),
        line=dict(width=3)
    ),
    secondary_y=True
)

fig.update_layout(
    title_text='Distribusi dan Tingkat Attrition Berdasarkan Kelompok Usia',
    template='plotly_white'
)

fig.update_xaxes(title_text='Kelompok Usia')
fig.update_yaxes(title_text='Jumlah Karyawan', secondary_y=False)
fig.update_yaxes(title_text='Tingkat Attrition (%)', secondary_y=True)

fig.show()
fig.write_image("./visualisasi/distribusi_usia_attrition.jpg", scale=1)

### Attrition by Departemen

In [ ]:
dept_attrition = df.groupby('Department')['Attrition'].mean().sort_values(ascending=False) * 100
dept_counts = df.groupby('Department').size()

dept_df = pd.DataFrame({
    'Jumlah Karyawan': dept_counts,
    'Tingkat Attrition (%)': dept_attrition
})

display(dept_df)

fig = px.bar(
    x=dept_attrition.index,
    y=dept_attrition.values,
    color=dept_attrition.values,
    text=dept_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Departemen',
    labels={'x': 'Departemen', 'y': 'Tingkat Attrition (%)'},
    color_continuous_scale='Viridis'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(coloraxis_showscale=False, template='plotly_white')

fig.show()

fig.write_image("./visualisasi/tingkat_attrition_departemen.jpg", scale=1)

### Attrition by JobLevel

In [ ]:
joblevel_attrition = df.groupby('JobLevel')['Attrition'].mean().sort_values(ascending=False) * 100
joblevel_counts = df.groupby('JobLevel').size()

joblevel_df = pd.DataFrame({
    'Tingkat Attrition (%)': joblevel_attrition,
    'Jumlah Karyawan': joblevel_counts
})

display(joblevel_df)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=joblevel_df.index,
    y=joblevel_df['Jumlah Karyawan'],
    name='Jumlah Karyawan',
    marker_color='lightblue',
    opacity=0.7,
    yaxis='y'
))

fig.add_trace(go.Scatter(
    x=joblevel_df.index,
    y=joblevel_df['Tingkat Attrition (%)'],
    name='Tingkat Attrition (%)',
    marker_color='red',
    mode='lines+markers',
    marker=dict(size=10),
    yaxis='y2'
))

fig.update_layout(
    title='Distribusi Attrition dan Jumlah Karyawan berdasarkan Job Level',
    xaxis=dict(title='Job Level'),
    yaxis=dict(title='Jumlah Karyawan', side='left', showgrid=False),
    yaxis2=dict(title='Tingkat Attrition (%)', side='right', overlaying='y', showgrid=False),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()

fig.write_image("./visualisasi/attrition_job_level.jpg", scale=2)

### Attrition by MaritalStatus

In [ ]:
marital_attrition = df.groupby('MaritalStatus')['Attrition'].mean() * 100
marital_counts = df.groupby('MaritalStatus').size()

marital_df = pd.DataFrame({
    'Jumlah Karyawan': marital_counts,
    'Tingkat Attrition (%)': marital_attrition
})

display(marital_df)

custom_labels = [f"{status}<br>Attrition: {attrition:.1f}%" 
                for status, attrition in zip(marital_attrition.index, marital_attrition.values)]

fig = go.Figure(go.Pie(
    labels=custom_labels,
    values=marital_counts.values,
    hole=0.4,
    textposition='outside',
    textinfo='percent', 
    pull=[0.05, 0.05, 0.05],
    marker=dict(
        colors=['#F9D876', '#F98B60', '#61D4D5'],
        line=dict(color='#FFFFFF', width=2)
    )
))

fig.update_layout(
    title='Distribusi Karyawan Berdasarkan Status Pernikahan',
    template='plotly_white',
    legend_title='Status Pernikahan & Tingkat Attrition',
    margin=dict(t=80, b=20, l=20, r=20),
    height=600,
    width=800
)

fig.add_annotation(
    text="Status<br>Pernikahan",
    x=0.5, y=0.5,
    font_size=14,
    showarrow=False
)

fig.show()

fig.write_image("./visualisasi/distribusi_status_pernikahan.jpg", scale=2)

### Attrition by Distance

In [ ]:
distance_stats = df.groupby('Attrition')['DistanceFromHome'].agg(['mean', 'median', 'min', 'max', 'std']).reset_index()

distance_stats['Attrition'] = distance_stats['Attrition'].map({0: 'Tidak Keluar', 1: 'Keluar'})

display(distance_stats)

fig = px.box(
    df, 
    x='Attrition', 
    y='DistanceFromHome',
    color='Attrition',
    title='Hubungan Jarak dari Rumah dengan Attrition',
    labels={'DistanceFromHome': 'Jarak dari Rumah (km)', 'Attrition': 'Status Keluar'},
    color_discrete_sequence=['#636EFA', '#EF553B']
)
fig.update_layout(template='plotly_white')

fig.show()

fig.write_image("./visualisasi/jarak_rumah_attrition.jpg", scale=2)

### Attrtion by Satisfaction

In [ ]:
df_satisfaction = df.copy()
for col in satisfaction_cols:
    df_satisfaction[col] = df_satisfaction[col].map(satisfaction_labels)

display(df_satisfaction)

fig = make_subplots(rows=2, cols=2, 
                   subplot_titles=['Kepuasan Lingkungan', 'Kepuasan Kerja', 
                                  'Kepuasan Hubungan', 'Work Life Balance'],)

col_positions = [
    ('EnvironmentSatisfaction', 1, 1),
    ('JobSatisfaction', 1, 2),
    ('RelationshipSatisfaction', 2, 1),
    ('WorkLifeBalance', 2, 2)
]

for col, row, col_pos in col_positions:
    attrition_by_satisfaction = df_satisfaction.groupby(col)['Attrition'].mean() * 100
    
    fig.add_trace(
        go.Bar(
            x=attrition_by_satisfaction.index,
            y=attrition_by_satisfaction.values,
            text=[f'{val:.1f}%' for val in attrition_by_satisfaction.values],
            textposition='auto',
            marker_color='lightseagreen'
        ),
        row=row, col=col_pos
    )

fig.update_layout(
    height=600,
    width=900,
    showlegend=False,
    title_text='Tingkat Attrition (%) Berdasarkan Faktor Kepuasan',
    template='plotly_white'
)

for i in fig['layout']['annotations']:
    i['font'] = dict(size=12, color='#000000')

fig.update_xaxes(title_text='Tingkat Kepuasan')
fig.update_yaxes(title_text='Tingkat Attrition (%)')

fig.show()

fig.write_image("./visualisasi/tingkat_attrition_kepuasan.jpg", scale=2)

### Attrition by JobRole

In [ ]:
jobrole_attrition = df.groupby('JobRole')['Attrition'].mean().sort_values(ascending=False) * 100
jobrole_counts = df.groupby('JobRole').size()

jobrole_df = pd.DataFrame({
    'Tingkat Attrition (%)': jobrole_attrition,
    'Jumlah Karyawan': jobrole_counts
})

display(jobrole_df)

fig = px.bar(
    x=jobrole_attrition.index,
    y=jobrole_attrition.values,
    color=jobrole_attrition.values,
    text=jobrole_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Job Role',
    labels={'x': 'Job Role', 'y': 'Tingkat Attrition (%)'},
    color_continuous_scale='Viridis'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    coloraxis_showscale=False,
    xaxis_tickangle=-45,
    template='plotly_white'
)
fig.show()

fig.write_image("./visualisasi/tingkat_attrition_job_role.jpg", scale=2)

### Attrition by Overtime

In [ ]:
overtime_attrition = df.groupby('OverTime')['Attrition'].mean() * 100
overtime_counts = df.groupby('OverTime').size()

overtime_df = pd.DataFrame({
    'Jumlah Karyawan': overtime_counts,
    'Tingkat Attrition (%)': overtime_attrition
})

display(overtime_df)

fig = px.bar(
    x=overtime_attrition.index,
    y=overtime_attrition.values,
    color=overtime_attrition.values,
    text=overtime_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Status Overtime',
    labels={'x': 'Overtime', 'y': 'Tingkat Attrition (%)'},
    color_continuous_scale='Reds'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(coloraxis_showscale=False, template='plotly_white')
fig.show()

fig.write_image("./visualisasi/tingkat_attrition_overtime.jpg", scale=2)

### Attrition by Salary Category

In [ ]:
salary_attrition = df.groupby('SalaryCategory')['Attrition'].mean() * 100
salary_counts = df.groupby('SalaryCategory').size()

order = ['Rendah (< 5000)', 'Sedang (5000-10000)', 'Tinggi (10000-15000)', 'Sangat Tinggi (>15000)']
salary_attrition = salary_attrition.reindex(order)
salary_counts = salary_counts.reindex(order)

salary_df = pd.DataFrame({
    'Jumlah Karyawan': salary_counts,
    'Tingkat Attrition (%)': salary_attrition
})

display(salary_df)

fig = px.bar(
    x=salary_attrition.index,
    y=salary_attrition.values,
    color=salary_attrition.values,
    text=salary_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Kategori Gaji',
    labels={'x': 'Kategori Gaji', 'y': 'Tingkat Attrition (%)'},
    color_continuous_scale='Greens_r'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(coloraxis_showscale=False, template='plotly_white')
fig.show()
fig.write_image("./visualisasi/tingkat_attrition_kategori_gaji.jpg", scale=2)

### Attrition by Gender

In [ ]:
gender_attrition = df.groupby('Gender')['Attrition'].mean() * 100
gender_counts = df.groupby('Gender').size()

gender_df = pd.DataFrame({
    'Jumlah Karyawan': gender_counts,
    'Tingkat Attrition (%)': gender_attrition
})

display(gender_df)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=gender_counts.index,
    y=gender_counts.values,
    name='Jumlah Karyawan',
    marker_color='skyblue',
    opacity=0.7,
    text=gender_counts.values,
    textposition='auto',
    yaxis='y'
))

fig.add_trace(go.Scatter(
    x=gender_attrition.index,
    y=gender_attrition.values,
    name='Tingkat Attrition (%)',
    marker_color='red',
    mode='lines+markers+text',
    marker=dict(size=12),
    text=[f'{val:.1f}%' for val in gender_attrition.values],
    textposition='top center',
    yaxis='y2'
))

fig.update_layout(
    title='Jumlah Karyawan dan Tingkat Attrition Berdasarkan Jenis Kelamin',
    xaxis=dict(title='Jenis Kelamin'),
    yaxis=dict(title='Jumlah Karyawan', side='left', showgrid=False),
    yaxis2=dict(title='Tingkat Attrition (%)', side='right', overlaying='y', showgrid=False),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()
fig.write_image("./visualisasi/tingkat_attrition_jenis_kelamin.jpg", scale=2)

### Attrition by Education Background

In [ ]:
education_field_attrition = df.groupby('EducationField')['Attrition'].mean() * 100
education_field_counts = df.groupby('EducationField').size()

education_field_df = pd.DataFrame({
    'Jumlah Karyawan': education_field_counts,
    'Tingkat Attrition (%)': education_field_attrition
})

education_field_df = education_field_df.sort_values('Tingkat Attrition (%)', ascending=False)
display(education_field_df)

fig = px.bar(
    education_field_df,
    x=education_field_df.index,
    y='Tingkat Attrition (%)',
    color='Tingkat Attrition (%)',
    text=education_field_df['Tingkat Attrition (%)'].round(1),
    title='Tingkat Attrition (%) Berdasarkan Bidang Pendidikan',
    labels={'y': 'Tingkat Attrition (%)', 'x': 'Bidang Pendidikan'},
    color_continuous_scale='Blues_r'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(coloraxis_showscale=False, template='plotly_white')
fig.show()
fig.write_image("./visualisasi/tingkat_attrition_bidang_pendidikan.jpg", scale=2)

### Attrition by LastTimePromoted

In [ ]:
promotion_attrition = df.groupby('PromotionCategory')['Attrition'].mean() * 100
promotion_counts = df.groupby('PromotionCategory').size()

order = ['Baru Dipromosikan', '1-2 Tahun', '3-5 Tahun', '> 5 Tahun']
promotion_attrition = promotion_attrition.reindex(order)
promotion_counts = promotion_counts.reindex(order)

promotion_df = pd.DataFrame({
    'Jumlah Karyawan': promotion_counts,
    'Tingkat Attrition (%)': promotion_attrition
})

display(promotion_df)

fig = px.line(
    x=promotion_attrition.index,
    y=promotion_attrition.values,
    markers=True,
    text=promotion_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Tahun Sejak Promosi Terakhir',
    labels={'x': 'Tahun Sejak Promosi Terakhir', 'y': 'Tingkat Attrition (%)'},
)
fig.update_traces(
    marker=dict(size=10, color='blue'),
    line=dict(width=3, color='purple'),
    textposition='top center'
)
fig.update_layout(template='plotly_white')
fig.show()
fig.write_image("./visualisasi/tingkat_attrition_promotion.jpg", scale=2)

## Multivariate

In [ ]:
# 1. Analisis multivariate: Overtime, Job Satisfaction, dan Attrition
# Membuat tabel pivot untuk melihat interaksi antara overtime dan tingkat kepuasan kerja
pivot_overtime_satisfaction = pd.crosstab(
    index=df['OverTime'],
    columns=df['JobSatisfaction'],
    values=df['Attrition'],
    aggfunc='mean'
) * 100

# Ubah nilai job satisfaction menjadi kategori untuk memudahkan pembacaan
satisfaction_labels = {1: 'Rendah', 2: 'Sedang', 3: 'Tinggi', 4: 'Sangat Tinggi'}
pivot_overtime_satisfaction.columns = [satisfaction_labels[col] for col in pivot_overtime_satisfaction.columns]

# Tampilkan dataframe
print("Tingkat Attrition (%) berdasarkan Overtime dan Job Satisfaction:")
display(pivot_overtime_satisfaction)

# Visualisasi dengan heatmap
fig = px.imshow(
    pivot_overtime_satisfaction,
    text_auto='.1f',
    aspect='auto',
    color_continuous_scale='RdYlGn_r',
    title='Tingkat Attrition (%) Berdasarkan Overtime dan Job Satisfaction',
    labels=dict(x='Tingkat Kepuasan Kerja', y='Overtime', color='Attrition Rate (%)')
)
fig.update_layout(template='plotly_white')
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_overtime_jobsatisfaction.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_overtime_jobsatisfaction.jpg'")

In [ ]:
# 2. Analisis multivariate: Department, Job Level, dan Attrition
# Membuat tabel pivot untuk melihat interaksi antara department dan job level
pivot_dept_joblevel = pd.crosstab(
    index=df['Department'],
    columns=df['JobLevel'],
    values=df['Attrition'],
    aggfunc='mean'
) * 100

# Tampilkan dataframe
print("Tingkat Attrition (%) berdasarkan Department dan Job Level:")
display(pivot_dept_joblevel)

# Visualisasi dengan heatmap
fig = px.imshow(
    pivot_dept_joblevel,
    text_auto='.1f',
    aspect='auto',
    color_continuous_scale='Viridis_r',
    title='Tingkat Attrition (%) Berdasarkan Department dan Job Level',
    labels=dict(x='Job Level', y='Department', color='Attrition Rate (%)')
)
fig.update_layout(template='plotly_white')
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_department_joblevel.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_department_joblevel.jpg'")

In [ ]:
# 3. Analisis multivariate: Gaji, Jarak dari Rumah, Overtime, dan Attrition
# Scatter plot dengan multiple dimensions
fig = px.scatter(
    df,
    x='MonthlyIncome',
    y='DistanceFromHome',
    color='Attrition',
    facet_col='OverTime',
    opacity=0.7,
    hover_name='JobRole',
    hover_data=['Age', 'YearsAtCompany', 'JobSatisfaction'],
    title='Hubungan antara Gaji, Jarak dari Rumah, Overtime dengan Attrition',
    labels={
        'MonthlyIncome': 'Gaji Bulanan',
        'DistanceFromHome': 'Jarak dari Rumah (km)',
        'Attrition': 'Status Attrition',
        'OverTime': 'Status Overtime'
    },
    color_discrete_sequence=['green', 'red']
)
fig.update_layout(
    template='plotly_white',
    height=600
)
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_income_distance_overtime.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_income_distance_overtime.jpg'")

In [ ]:
# 4. Analisis multivariate: Usia, Job Level, dan Attrition
# Buat kategori usia
if 'AgeGroup' not in df.columns:
    df['AgeGroup'] = pd.cut(
        df['Age'],
        bins=[20, 30, 40, 50, 60],
        labels=['20-29', '30-39', '40-49', '50-59']
    )

# Buat tabel pivot
pivot_age_joblevel = pd.crosstab(
    index=df['AgeGroup'],
    columns=df['JobLevel'],
    values=df['Attrition'],
    aggfunc='mean'
) * 100

# Tampilkan dataframe
print("Tingkat Attrition (%) berdasarkan Kelompok Usia dan Job Level:")
display(pivot_age_joblevel)

# Visualisasi dengan heatmap
fig = px.imshow(
    pivot_age_joblevel,
    text_auto='.1f',
    aspect='auto',
    color_continuous_scale='Plasma_r',
    title='Tingkat Attrition (%) Berdasarkan Kelompok Usia dan Job Level',
    labels=dict(x='Job Level', y='Kelompok Usia', color='Attrition Rate (%)')
)
fig.update_layout(template='plotly_white')
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_age_joblevel.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_age_joblevel.jpg'")

In [ ]:
# Analisis kombinasi overtime dan work-life balance
cross_analysis = pd.crosstab(
    index=df['OverTime'],
    columns=df['WorkLifeBalance'],
    values=df['Attrition'],
    aggfunc='mean'
) * 100

# Ubah nilai worklife balance menjadi label
cross_analysis.columns = ['Buruk', 'Sedang', 'Baik', 'Sangat Baik']

display(cross_analysis)

# Visualisasi dengan heatmap
fig = px.imshow(
    cross_analysis,
    text_auto='.1f',
    aspect='auto',
    color_continuous_scale='RdYlGn_r',
    title='Tingkat Attrition (%) Berdasarkan Overtime dan Work-Life Balance',
    labels=dict(x='Work-Life Balance', y='Overtime', color='Attrition Rate (%)')
)
fig.update_layout(template='plotly_white')
fig.show()

In [ ]:
# 5. Analisis multivariate: 3D Scatter Plot - Gaji, Tahun di Perusahaan, Jarak dari Rumah, dan Attrition
fig = px.scatter_3d(
    df,
    x='MonthlyIncome',
    y='YearsAtCompany',
    z='DistanceFromHome',
    color='Attrition',
    opacity=0.7,
    size='JobLevel',
    hover_name='JobRole',
    title='Hubungan 3D antara Gaji, Tahun Bekerja, Jarak dari Rumah, dan Attrition',
    labels={
        'MonthlyIncome': 'Gaji Bulanan',
        'YearsAtCompany': 'Tahun di Perusahaan',
        'DistanceFromHome': 'Jarak dari Rumah (km)',
        'Attrition': 'Status Attrition',
        'JobLevel': 'Level Pekerjaan'
    },
    color_discrete_sequence=['green', 'red']
)
fig.update_layout(
    template='plotly_white',
    scene=dict(
        xaxis_title='Gaji Bulanan',
        yaxis_title='Tahun di Perusahaan',
        zaxis_title='Jarak dari Rumah (km)'
    )
)
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_3d_income_years_distance.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_3d_income_years_distance.jpg'")

In [ ]:
# 6. Analisis multivariate: Parallel Coordinates Plot untuk melihat banyak dimensi sekaligus
# Pilih kolom numerik yang relevan
cols_for_parallel = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'YearsAtCompany', 
                     'WorkLifeBalance', 'JobSatisfaction', 'JobLevel']

# Buat Parallel Coordinates Plot
fig = px.parallel_coordinates(
    df, 
    color='Attrition',
    dimensions=cols_for_parallel,
    color_continuous_scale=px.colors.diverging.Tealrose,
    title='Parallel Coordinates Plot untuk Berbagai Dimensi dan Attrition'
)
fig.update_layout(
    template='plotly_white',
    height=600
)
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_parallel_coordinates.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_parallel_coordinates.jpg'")

In [ ]:
# 7. Analisis multivariate: Bubble Chart untuk Department, Job Role, Job Satisfaction dan Attrition
# Hitung tingkat attrition berdasarkan kombinasi department dan job role
attrition_by_dept_role = df.groupby(['Department', 'JobRole'])['Attrition'].mean() * 100
count_by_dept_role = df.groupby(['Department', 'JobRole']).size()
satisfaction_by_dept_role = df.groupby(['Department', 'JobRole'])['JobSatisfaction'].mean()

# Buat dataframe
multivariate_df = pd.DataFrame({
    'Tingkat Attrition (%)': attrition_by_dept_role,
    'Jumlah Karyawan': count_by_dept_role,
    'Rata-rata Kepuasan Kerja': satisfaction_by_dept_role
}).reset_index()

# Tampilkan dataframe
print("Tingkat Attrition (%), Jumlah Karyawan, dan Rata-rata Kepuasan Kerja berdasarkan Department dan Job Role:")
display(multivariate_df.sort_values('Tingkat Attrition (%)', ascending=False))

# Visualisasi dengan bubble chart
fig = px.scatter(
    multivariate_df,
    x='Rata-rata Kepuasan Kerja',
    y='Tingkat Attrition (%)',
    size='Jumlah Karyawan',
    color='Department',
    hover_name='JobRole',
    text='JobRole',
    title='Tingkat Attrition (%) vs Kepuasan Kerja berdasarkan Department dan Job Role',
    labels={
        'Rata-rata Kepuasan Kerja': 'Rata-rata Kepuasan Kerja (1-4)',
        'Tingkat Attrition (%)': 'Tingkat Attrition (%)'
    }
)
fig.update_traces(
    textposition='top center',
    marker=dict(sizemode='area', sizeref=0.1)
)
fig.update_layout(
    template='plotly_white',
    height=800,
    width=1000
)
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_dept_role_satisfaction.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_dept_role_satisfaction.jpg'")

In [ ]:
# Pastikan semua feature engineering dan variabel baru yang ingin disimpan sudah dibuat

# Buat daftar variabel baru yang telah dibuat
new_variables = [
    'SalaryCategory',        # Kategori gaji
    'PromotionCategory',     # Kategori tahun sejak promosi terakhir
    'AgeGroup',              # Kelompok usia
    'DistanceCategory'       # Kategori jarak dari rumah
]

# Periksa variabel mana yang sudah ada di dataframe
existing_variables = [var for var in new_variables if var in df.columns]

# Jika ada variabel yang belum dibuat, buat terlebih dahulu
if 'SalaryCategory' not in existing_variables:
    df['SalaryCategory'] = df['MonthlyIncome'].apply(lambda income: 
        'Rendah (< 5000)' if income < 5000 else
        'Sedang (5000-10000)' if income < 10000 else
        'Tinggi (10000-15000)' if income < 15000 else
        'Sangat Tinggi (>15000)'
    )

if 'PromotionCategory' not in existing_variables:
    df['PromotionCategory'] = df['YearsSinceLastPromotion'].apply(lambda years:
        'Baru Dipromosikan' if years == 0 else
        '1-2 Tahun' if years <= 2 else
        '3-5 Tahun' if years <= 5 else
        '> 5 Tahun'
    )

if 'AgeGroup' not in existing_variables:
    df['AgeGroup'] = df['Age'].apply(lambda age:
        '< 30' if age < 30 else
        '30-39' if age < 40 else
        '40-49' if age < 50 else
        '50+'
    )

if 'DistanceCategory' not in existing_variables:
    df['DistanceCategory'] = pd.cut(
        df['DistanceFromHome'],
        bins=[0, 5, 10, 20, 30],
        labels=['0-5 km', '6-10 km', '11-20 km', '21-30 km']
    )

# Tampilkan kolom-kolom yang ada di data final
print("Kolom-kolom dalam dataset final:")
display(pd.DataFrame({'Nama Kolom': df.columns}))

# Simpan data final
df.to_csv('./data/data_final_employee.csv', index=False)

# Jika ingin menyimpan dalam format Excel
try:
    df.to_excel('./data/data_final_employee.xlsx', index=False)
except Exception as e:
    print(f"Gagal menyimpan dalam format Excel: {e}")

# Segmentation Using Unsupervised Learning

In [10]:
CB_COLORS = {
    'blue': '#648FFF',    # biru
    'orange': '#FE6100',  # oranye
    'green': '#009E73',   # hijau
    'pink': '#F564E3',    # pink
    'brown': '#B79762',   # coklat
    'teal': '#50B4E9',    # teal
    'red': '#D62728',     # merah
    'grey': '#999999',    # abu-abu
}

# Palette untuk high risk vs low risk
HIGH_RISK_COLOR = CB_COLORS['orange']   # Oranye untuk risiko tinggi
LOW_RISK_COLOR = CB_COLORS['blue']      # Biru untuk risiko rendah

import warnings
warnings.filterwarnings('ignore')

# Fungsi untuk mencetak header
def print_header(text):
    print(f"\n{'='*80}\n{text}\n{'='*80}")

# Fungsi untuk menampilkan dan menyimpan gambar
def display_and_save_figure(fig, filename, show_fig=True):
    # Tampilkan gambar di output cell
    if show_fig:
        fig.show()
    
    # Simpan gambar di lokal
    fig.write_image(filename)
    print(f"Visualisasi disimpan sebagai '{filename}'")
    
    return fig

In [11]:
# 1. Baca dataset
print_header("MEMBACA DAN MEMPERSIAPKAN DATA")
df = pd.read_csv('./data/data_final_employee.csv')
print(f"Ukuran dataset: {df.shape}")


MEMBACA DAN MEMPERSIAPKAN DATA
Ukuran dataset: (1058, 39)


In [12]:
# 2. Tangani missing value
print("\nMenangani missing value...")
missing_values = df.isnull().sum()
if missing_values.sum() > 0:
    print(f"Missing values ditemukan: {missing_values[missing_values > 0]}")
    df.dropna(inplace=True)
    print(f"Ukuran dataset setelah menghapus missing value: {df.shape}")
else:
    print("Tidak ada missing value ditemukan.")


Menangani missing value...
Tidak ada missing value ditemukan.


In [13]:
# 3. Pastikan kolom Attrition numerik (0/1)
if df['Attrition'].dtype == 'object':
    df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
    print("\nKolom Attrition dikonversi dari kategorikal ke numerik (1/0)")
elif df['Attrition'].max() > 1:
    df['Attrition'] = (df['Attrition'] > 0).astype(int)
    print("\nKolom Attrition dikonversi ke binary (0/1)")

In [14]:
# 4. Feature Engineering
print_header("FEATURE ENGINEERING")

# Pilih fitur numerik yang relevan
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_features = [f for f in numerical_features if f != 'Attrition' and f != 'EmployeeId' and f != 'EmployeeCount' and f != 'StandardHours' and f != 'Cluster']

print(f"\nMenggunakan {len(numerical_features)} fitur numerik asli")

# Buat fitur engineering baru
print("\nMembuat fitur gabungan dan rasio...")

# Cek fitur yang tersedia
available_features = set(df.columns)

# Feature engineering: Buat fitur baru
# 1. Rasio Compensation ke Job Level
if 'MonthlyIncome' in available_features and 'JobLevel' in available_features:
    df['SalaryPerLevel'] = df['MonthlyIncome'] / df['JobLevel'].replace(0, 1)
    print("- Fitur baru: SalaryPerLevel (Gaji per level jabatan)")

# 2. Indeks Kepuasan Kerja Gabungan
satisfaction_columns = [col for col in ['JobSatisfaction', 'EnvironmentSatisfaction', 
                                      'WorkLifeBalance', 'RelationshipSatisfaction'] 
                       if col in available_features]

if len(satisfaction_columns) >= 2:
    df['SatisfactionIndex'] = df[satisfaction_columns].mean(axis=1)
    print(f"- Fitur baru: SatisfactionIndex (Rata-rata dari {len(satisfaction_columns)} fitur kepuasan)")

# 3. Rasio Promosi (tahun sejak promosi dibagi masa kerja)
if 'YearsSinceLastPromotion' in available_features and 'YearsAtCompany' in available_features:
    df['PromotionRatio'] = df['YearsSinceLastPromotion'] / df['YearsAtCompany'].replace(0, 1)
    print("- Fitur baru: PromotionRatio (Rasio tahun sejak promosi terhadap masa kerja)")

# Update list fitur numerik dengan fitur baru
engineered_features = [col for col in df.columns if col not in numerical_features 
                      and col != 'Attrition' and col != 'EmployeeId' 
                      and col != 'EmployeeCount' and col != 'StandardHours'
                      and col != 'Cluster'
                      and df[col].dtype != 'object']

print(f"\nMenambahkan {len(engineered_features)} fitur hasil feature engineering:")
print(engineered_features)

# Gabungkan fitur asli dan fitur baru
all_features = numerical_features + engineered_features
print(f"\nTotal fitur untuk evaluasi: {len(all_features)}")



FEATURE ENGINEERING

Menggunakan 23 fitur numerik asli

Membuat fitur gabungan dan rasio...
- Fitur baru: SalaryPerLevel (Gaji per level jabatan)
- Fitur baru: SatisfactionIndex (Rata-rata dari 4 fitur kepuasan)
- Fitur baru: PromotionRatio (Rasio tahun sejak promosi terhadap masa kerja)

Menambahkan 3 fitur hasil feature engineering:
['SalaryPerLevel', 'SatisfactionIndex', 'PromotionRatio']

Total fitur untuk evaluasi: 26


In [15]:
# 5. Feature Selection
print_header("FEATURE SELECTION")

# Persiapkan data
X_all = df[all_features]
y = df['Attrition']

# Standarisasi semua fitur
print("\nStandardisasi data...")
robust_scaler = RobustScaler()  # Robust terhadap outlier
X_scaled = robust_scaler.fit_transform(X_all)
X_scaled_df = pd.DataFrame(X_scaled, columns=all_features)

# Random Forest Importance
print("\nMencari feature importance dengan Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(X_scaled, y)

rf_importance = pd.DataFrame({
    'Feature': all_features,
    'RF_Importance': rf.feature_importances_
}).sort_values(by='RF_Importance', ascending=False)

print(rf_importance.head(10))

# Visualization of feature importance with Plotly
fig = px.bar(
    rf_importance.head(10),
    x='RF_Importance',
    y='Feature',
    orientation='h',
    title='Top 10 Fitur Terpenting untuk Attrition',
    color='RF_Importance',
    color_continuous_scale=['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000']  # Colorblind friendly scale
)
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    coloraxis_showscale=False,
    margin=dict(l=20, r=20, t=40, b=20),
    font=dict(size=12),
    height=500,
    width=800
)

# Tampilkan dan simpan visualisasi feature importance
display_and_save_figure(fig, "feature_importance_colorblind.png")

# Pilih top 10 fitur
top_features = rf_importance.head(10)['Feature'].tolist()
print(f"\nMenggunakan 10 fitur terbaik untuk clustering: {top_features}")



FEATURE SELECTION

Standardisasi data...

Mencari feature importance dengan Random Forest...
              Feature  RF_Importance
9       MonthlyIncome       0.074549
0                 Age       0.067456
10        MonthlyRate       0.060989
23     SalaryPerLevel       0.058753
15   StockOptionLevel       0.056937
1           DailyRate       0.056510
16  TotalWorkingYears       0.055500
24  SatisfactionIndex       0.054107
5          HourlyRate       0.052794
2    DistanceFromHome       0.041263


Visualisasi disimpan sebagai 'feature_importance_colorblind.png'

Menggunakan 10 fitur terbaik untuk clustering: ['MonthlyIncome', 'Age', 'MonthlyRate', 'SalaryPerLevel', 'StockOptionLevel', 'DailyRate', 'TotalWorkingYears', 'SatisfactionIndex', 'HourlyRate', 'DistanceFromHome']


In [16]:
# 6. Transformasi Data Advanced
print_header("TRANSFORMASI DATA ADVANCED")

# Pilih subset data dengan fitur terbaik
X_top = X_scaled_df[top_features]
print(f"Data untuk clustering: {X_top.shape}")

# Power Transformation untuk menormalkan distribusi
print("\nMenerapkan Power Transformation untuk normalisasi...")
power_transformer = PowerTransformer(method='yeo-johnson')
X_transformed = power_transformer.fit_transform(X_top)
print("Transformasi selesai.")



TRANSFORMASI DATA ADVANCED
Data untuk clustering: (1058, 10)

Menerapkan Power Transformation untuk normalisasi...
Transformasi selesai.


In [17]:
# 7. Dimensionality Reduction
print_header("DIMENSIONALITY REDUCTION")

# UMAP untuk visualisasi dan segmentasi yang lebih baik
print("\nMenerapkan UMAP untuk reduksi dimensi...")
umap_reducer = UMAP(n_components=2, random_state=42, min_dist=0.1, n_neighbors=15)
X_umap = umap_reducer.fit_transform(X_transformed)
print(f"Dimensi setelah UMAP: {X_umap.shape}")

# Visualisasi hasil UMAP berdasarkan attrition aktual dengan Plotly (colorblind friendly)
umap_df = pd.DataFrame(X_umap, columns=['UMAP_1', 'UMAP_2'])
umap_df['Attrition'] = y
umap_df['Attrition_Label'] = umap_df['Attrition'].map({0: 'Tidak Keluar', 1: 'Keluar'})

fig = px.scatter(
    umap_df, 
    x='UMAP_1', 
    y='UMAP_2', 
    color='Attrition_Label',
    color_discrete_map={
        'Keluar': HIGH_RISK_COLOR,
        'Tidak Keluar': LOW_RISK_COLOR
    },
    title='Visualisasi Data dengan UMAP (warna berdasarkan Attrition Aktual)',
    opacity=0.7,
    size_max=10,
    width=900,
    height=700
)
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    legend_title_text='Status',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    font=dict(size=12)
)

# Tampilkan dan simpan visualisasi UMAP
display_and_save_figure(fig, "umap_visualization_actual_colorblind.png")



DIMENSIONALITY REDUCTION

Menerapkan UMAP untuk reduksi dimensi...
Dimensi setelah UMAP: (1058, 2)


Visualisasi disimpan sebagai 'umap_visualization_actual_colorblind.png'


In [18]:
# 8. Clustering dengan 2 cluster (Risiko Rendah dan Tinggi)
print_header("CLUSTERING DENGAN 2 CLUSTER: RISIKO RENDAH DAN RISIKO TINGGI")

# Pilih antara beberapa metode clustering
clustering_methods = {
    'KMeans': KMeans(n_clusters=2, random_state=42, n_init=20),
    'GaussianMixture': GaussianMixture(n_components=2, random_state=42, covariance_type='full')
}

# Evaluasi silhouette score untuk kedua metode
silhouette_scores = {}
for method_name, model in clustering_methods.items():
    if method_name == 'GaussianMixture':
        model.fit(X_umap)
        labels = model.predict(X_umap)
    else:
        labels = model.fit_predict(X_umap)
    
    score = silhouette_score(X_umap, labels)
    silhouette_scores[method_name] = score
    print(f"{method_name} silhouette score: {score:.3f}")

# Pilih metode terbaik berdasarkan silhouette score
best_method = max(silhouette_scores, key=silhouette_scores.get)
print(f"\nMetode terbaik: {best_method} dengan silhouette score: {silhouette_scores[best_method]:.3f}")

# Aplikasikan metode terbaik
if best_method == 'GaussianMixture':
    best_model = clustering_methods[best_method]
    best_model.fit(X_umap)
    cluster_labels = best_model.predict(X_umap)
    
    # Probabilitas keanggotaan cluster untuk GMM
    cluster_probs = best_model.predict_proba(X_umap)
    df['ClusterUncertainty'] = 1 - np.max(cluster_probs, axis=1)
else:  # KMeans
    best_model = clustering_methods[best_method]
    cluster_labels = best_model.fit_predict(X_umap)

# Tambahkan label cluster ke dataframe
df['Cluster'] = cluster_labels


CLUSTERING DENGAN 2 CLUSTER: RISIKO RENDAH DAN RISIKO TINGGI
KMeans silhouette score: 0.483
GaussianMixture silhouette score: 0.481

Metode terbaik: KMeans dengan silhouette score: 0.483


In [19]:
# 9. Analisis tingkat attrition per cluster
attrition_by_cluster = df.groupby('Cluster')['Attrition'].mean() * 100
cluster_sizes = df.groupby('Cluster').size()

print("\nAnalisis tingkat attrition per cluster:")
for cluster in sorted(attrition_by_cluster.index):
    print(f"Cluster {cluster}: {cluster_sizes[cluster]} karyawan, tingkat attrition {attrition_by_cluster[cluster]:.2f}%")


Analisis tingkat attrition per cluster:
Cluster 0: 472 karyawan, tingkat attrition 25.64%
Cluster 1: 586 karyawan, tingkat attrition 9.90%


In [20]:
# 10. Tentukan cluster dengan risiko tinggi
high_risk_cluster = attrition_by_cluster.idxmax()
low_risk_cluster = attrition_by_cluster.idxmin()

print(f"\nCluster dengan risiko tinggi: Cluster {high_risk_cluster} ({attrition_by_cluster[high_risk_cluster]:.2f}%)")
print(f"Cluster dengan risiko rendah: Cluster {low_risk_cluster} ({attrition_by_cluster[low_risk_cluster]:.2f}%)")


Cluster dengan risiko tinggi: Cluster 0 (25.64%)
Cluster dengan risiko rendah: Cluster 1 (9.90%)


In [21]:
# 11. Buat kolom Risk Level dengan 2 kategori saja
df['RiskLevel'] = df['Cluster'].map({
    high_risk_cluster: 'Risiko Tinggi', 
    low_risk_cluster: 'Risiko Rendah'
})

In [22]:
# 12. Hitung distribusi risiko
risk_distribution = df['RiskLevel'].value_counts()
risk_percentage = df['RiskLevel'].value_counts(normalize=True) * 100

print("\nDistribusi Karyawan berdasarkan Level Risiko Attrition:")
for risk, count in risk_distribution.items():
    percentage = risk_percentage[risk]
    print(f"{risk}: {count} karyawan ({percentage:.1f}%)")


Distribusi Karyawan berdasarkan Level Risiko Attrition:
Risiko Rendah: 586 karyawan (55.4%)
Risiko Tinggi: 472 karyawan (44.6%)


In [23]:
# 13. Visualisasi distribusi risiko dengan Plotly (colorblind friendly)
risk_df = pd.DataFrame({
    'Level Risiko': risk_distribution.index,
    'Jumlah Karyawan': risk_distribution.values,
    'Persentase': risk_percentage.values
})

fig = px.bar(
    risk_df,
    x='Level Risiko',
    y='Jumlah Karyawan',
    color='Level Risiko',
    color_discrete_map={
        'Risiko Tinggi': HIGH_RISK_COLOR,
        'Risiko Rendah': LOW_RISK_COLOR
    },
    text=[f"{count} ({pct:.1f}%)" for count, pct in zip(risk_df['Jumlah Karyawan'], risk_df['Persentase'])],
    title='Distribusi Karyawan berdasarkan Level Risiko Attrition',
    height=500,
    width=800
)
fig.update_traces(textposition='inside', textfont=dict(color='white', size=14))
fig.update_layout(
    xaxis_title='Level Risiko',
    yaxis_title='Jumlah Karyawan',
    font=dict(size=12)
)

# Tampilkan dan simpan visualisasi distribusi risiko
display_and_save_figure(fig, "risk_distribution_binary_colorblind.png")

Visualisasi disimpan sebagai 'risk_distribution_binary_colorblind.png'


In [24]:
# 14. Visualisasi segmentasi dengan UMAP menggunakan Plotly (colorblind friendly)
umap_cluster_df = pd.DataFrame(X_umap, columns=['UMAP_1', 'UMAP_2'])
umap_cluster_df['Risk Level'] = df['RiskLevel']

fig = px.scatter(
    umap_cluster_df, 
    x='UMAP_1', 
    y='UMAP_2', 
    color='Risk Level',
    color_discrete_map={
        'Risiko Tinggi': HIGH_RISK_COLOR,
        'Risiko Rendah': LOW_RISK_COLOR
    },
    title='Segmentasi Risiko Attrition: Risiko Tinggi vs Risiko Rendah',
    opacity=0.7,
    width=900,
    height=700
)
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    legend_title_text='Level Risiko',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    font=dict(size=12)
)

# Tampilkan dan simpan visualisasi segmentasi
display_and_save_figure(fig, "binary_risk_segmentation_colorblind.png")

Visualisasi disimpan sebagai 'binary_risk_segmentation_colorblind.png'


In [25]:
# 15. Analisis profil untuk setiap level risiko
print_header("ANALISIS PROFIL RISIKO")

# Pilih fitur penting untuk analisis profil
profile_features = top_features[:6]  # Gunakan 6 fitur teratas untuk profil
profile_features = list(set(profile_features))  # Hapus duplikat

# Hitung profil rata-rata
profile_by_risk = df.groupby('RiskLevel')[profile_features].mean()

print("\nProfil rata-rata berdasarkan level risiko:")
print(profile_by_risk)

# Hitung persentase perbedaan antara risiko tinggi vs rendah
high_profile = profile_by_risk.loc['Risiko Tinggi'] if 'Risiko Tinggi' in profile_by_risk.index else profile_by_risk.iloc[0]
low_profile = profile_by_risk.loc['Risiko Rendah'] if 'Risiko Rendah' in profile_by_risk.index else profile_by_risk.iloc[1]

# Hitung perbedaan persentase
pct_diff = (high_profile - low_profile) / low_profile * 100

print("\nPerbedaan persentase (Risiko Tinggi vs Risiko Rendah):")
for feature, diff in pct_diff.items():
    print(f"{feature}: {diff:+.1f}%")


ANALISIS PROFIL RISIKO

Profil rata-rata berdasarkan level risiko:
               SalaryPerLevel        Age   MonthlyRate   DailyRate  \
RiskLevel                                                            
Risiko Rendah     3020.368629  37.547782  13989.883959  825.015358   
Risiko Tinggi     2932.708651  36.444915  14611.677966  790.332627   

               StockOptionLevel  MonthlyIncome  
RiskLevel                                       
Risiko Rendah          1.372014    6940.064846  
Risiko Tinggi          0.006356    6235.957627  

Perbedaan persentase (Risiko Tinggi vs Risiko Rendah):
SalaryPerLevel: -2.9%
Age: -2.9%
MonthlyRate: +4.4%
DailyRate: -4.2%
StockOptionLevel: -99.5%
MonthlyIncome: -10.1%


In [26]:
# 16. Visualisasi perbedaan profil dengan Plotly (colorblind friendly)
print("\nMembuat visualisasi perbedaan profil...")

# Pilih top 8 fitur dengan perbedaan terbesar untuk visualisasi
top_diff_features = pct_diff.abs().sort_values(ascending=False).head(8).index.tolist()
diff_df = pd.DataFrame({
    'Fitur': top_diff_features,
    'Perbedaan (%)': pct_diff[top_diff_features].values
})

# Gunakan colorscale yang menunjukkan perbedaan positif/negatif
colors = [HIGH_RISK_COLOR if val > 0 else LOW_RISK_COLOR for val in diff_df['Perbedaan (%)']]

fig = px.bar(
    diff_df,
    x='Fitur',
    y='Perbedaan (%)',
    color='Perbedaan (%)',
    color_continuous_scale=[LOW_RISK_COLOR, '#FFFFFF', HIGH_RISK_COLOR],
    text=[f"{val:+.1f}%" for val in diff_df['Perbedaan (%)']],
    title='Perbedaan Karakteristik Risiko Tinggi vs Risiko Rendah (%)',
    height=600,
    width=900
)
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Fitur',
    yaxis_title='Perbedaan Persentase (%)',
    xaxis={'categoryorder': 'total ascending'},
    font=dict(size=12)
)
# Tambahkan garis horizontal di 0
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=len(top_diff_features)-0.5,
    y1=0,
    line=dict(color="black", width=1, dash="dash")
)

# Tampilkan dan simpan visualisasi perbedaan profil
display_and_save_figure(fig, "profile_differences_colorblind.png")


Membuat visualisasi perbedaan profil...


Visualisasi disimpan sebagai 'profile_differences_colorblind.png'


In [27]:
# 17. Analisis distribusi departemen per level risiko dengan Plotly
if 'Department' in df.columns:
    dept_by_risk = pd.crosstab(df['RiskLevel'], df['Department'], normalize='index') * 100
    
    print("\nDistribusi Department (%) per Level Risiko:")
    print(dept_by_risk)
    
    # Reshape data untuk Plotly
    dept_data = []
    for risk_level in dept_by_risk.index:
        for dept in dept_by_risk.columns:
            dept_data.append({
                'RiskLevel': risk_level,
                'Department': dept,
                'Percentage': dept_by_risk.loc[risk_level, dept]
            })
    
    dept_df = pd.DataFrame(dept_data)
    
    fig = px.bar(
        dept_df,
        x='RiskLevel',
        y='Percentage',
        color='Department',
        color_discrete_sequence=px.colors.qualitative.Bold,  # Colorblind friendly
        title='Distribusi Department berdasarkan Level Risiko',
        height=600,
        width=900,
        barmode='stack'
    )
    fig.update_layout(
        xaxis_title='Level Risiko',
        yaxis_title='Persentase (%)',
        legend_title='Department',
        font=dict(size=12)
    )
    
    # Tampilkan dan simpan visualisasi distribusi department
    display_and_save_figure(fig, "department_by_risk_binary_colorblind.png")


Distribusi Department (%) per Level Risiko:
Department     Human Resources  Research & Development      Sales
RiskLevel                                                        
Risiko Rendah         3.924915               66.894198  29.180887
Risiko Tinggi         3.177966               65.466102  31.355932


Visualisasi disimpan sebagai 'department_by_risk_binary_colorblind.png'


In [28]:
# 18. Analisis distribusi job role per level risiko dengan Plotly
if 'JobRole' in df.columns:
    role_by_risk = pd.crosstab(df['RiskLevel'], df['JobRole'], normalize='index') * 100
    
    print("\nDistribusi Job Role (%) per Level Risiko:")
    print(role_by_risk)
    
    # Reshape data untuk Plotly
    role_data = []
    for risk_level in role_by_risk.index:
        for role in role_by_risk.columns:
            role_data.append({
                'RiskLevel': risk_level,
                'JobRole': role,
                'Percentage': role_by_risk.loc[risk_level, role]
            })
    
    role_df = pd.DataFrame(role_data)
    
    fig = px.bar(
        role_df,
        x='RiskLevel',
        y='Percentage',
        color='JobRole',
        color_discrete_sequence=px.colors.qualitative.Plotly,  # Colorblind friendly
        title='Distribusi Job Role berdasarkan Level Risiko',
        height=600,
        width=900,
        barmode='stack'
    )
    fig.update_layout(
        xaxis_title='Level Risiko',
        yaxis_title='Persentase (%)',
        legend_title='Job Role',
        font=dict(size=12)
    )
    
    # Tampilkan dan simpan visualisasi distribusi job role
    display_and_save_figure(fig, "jobrole_by_risk_binary_colorblind.png")


Distribusi Job Role (%) per Level Risiko:
JobRole        Healthcare Representative  Human Resources  \
RiskLevel                                                   
Risiko Rendah                   8.191126         2.901024   
Risiko Tinggi                   8.474576         2.754237   

JobRole        Laboratory Technician   Manager  Manufacturing Director  \
RiskLevel                                                                
Risiko Rendah              18.259386  9.044369               10.580205   
Risiko Tinggi              17.161017  5.508475                9.533898   

JobRole        Research Director  Research Scientist  Sales Executive  \
RiskLevel                                                               
Risiko Rendah           6.143345           18.771331        21.843003   
Risiko Tinggi           5.508475           22.033898        22.033898   

JobRole        Sales Representative  
RiskLevel                            
Risiko Rendah              4.266212  
Risiko T

Visualisasi disimpan sebagai 'jobrole_by_risk_binary_colorblind.png'


In [29]:
# 19. Dashboard ringkasan dengan Plotly
print_header("DASHBOARD RINGKASAN")

# Create dashboard with 2x2 subplots
fig = make_subplots(
    rows=2, 
    cols=2,
    subplot_titles=(
        'Segmentasi Risiko Attrition',
        'Distribusi Karyawan berdasarkan Risiko',
        'Top 5 Perbedaan Karakteristik',
        'Distribusi Department per Risiko'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "bar"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

# 1. Segmentasi (Tinggi vs Rendah)
fig.add_trace(
    go.Scatter(
        x=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Tinggi']['UMAP_1'],
        y=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Tinggi']['UMAP_2'],
        mode='markers',
        marker=dict(color=HIGH_RISK_COLOR, size=8),
        name='Risiko Tinggi'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Rendah']['UMAP_1'],
        y=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Rendah']['UMAP_2'],
        mode='markers',
        marker=dict(color=LOW_RISK_COLOR, size=8),
        name='Risiko Rendah'
    ),
    row=1, col=1
)

# 2. Distribusi risiko
for i, risk in enumerate(risk_df['Level Risiko']):
    color = HIGH_RISK_COLOR if risk == 'Risiko Tinggi' else LOW_RISK_COLOR
    fig.add_trace(
        go.Bar(
            x=[risk],
            y=[risk_df['Jumlah Karyawan'].iloc[i]],
            text=[f"{risk_df['Jumlah Karyawan'].iloc[i]} ({risk_df['Persentase'].iloc[i]:.1f}%)"],
            textposition='inside',
            name=risk,
            marker_color=color,
            showlegend=False
        ),
        row=1, col=2
    )

# 3. Top 5 perbedaan karakteristik
top_5_diff = pct_diff.abs().sort_values(ascending=False).head(5)
for feature in top_5_diff.index:
    color = HIGH_RISK_COLOR if pct_diff[feature] > 0 else LOW_RISK_COLOR
    fig.add_trace(
        go.Bar(
            x=[feature],
            y=[pct_diff[feature]],
            text=[f"{pct_diff[feature]:+.1f}%"],
            textposition='outside',
            marker_color=color,
            showlegend=False,
            name=feature
        ),
        row=2, col=1
    )

# 4. Department atau Job Role distribution
if 'Department' in df.columns:
    # Ambil 3 department dengan persentase tertinggi di setiap level risiko
    for risk_level in dept_by_risk.index:
        top_depts = dept_by_risk.loc[risk_level].sort_values(ascending=False).head(3)
        for dept, pct in top_depts.items():
            color = HIGH_RISK_COLOR if risk_level == 'Risiko Tinggi' else LOW_RISK_COLOR
            fig.add_trace(
                go.Bar(
                    x=[dept],
                    y=[pct],
                    text=[f"{pct:.1f}%"],
                    textposition='outside',
                    name=f"{risk_level} - {dept}",
                    marker_color=color,
                    opacity=0.7,
                    showlegend=False
                ),
                row=2, col=2
            )

# Update layout
fig.update_layout(
    title_text='DASHBOARD SEGMENTASI RISIKO ATTRITION KARYAWAN',
    height=900,
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(size=12)
)

# Update axes
fig.update_xaxes(title_text='UMAP Dimension 1', row=1, col=1)
fig.update_yaxes(title_text='UMAP Dimension 2', row=1, col=1)

fig.update_xaxes(title_text='Level Risiko', row=1, col=2)
fig.update_yaxes(title_text='Jumlah Karyawan', row=1, col=2)

fig.update_xaxes(title_text='Fitur', row=2, col=1)
fig.update_yaxes(title_text='Perbedaan (%)', row=2, col=1)

fig.update_xaxes(title_text='Department', row=2, col=2)
fig.update_yaxes(title_text='Persentase (%)', row=2, col=2)

# Add horizontal line at y=0 for differences
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=4.5,
    y1=0,
    line=dict(color="black", width=1, dash="dash"),
    row=2, col=1
)

# Tampilkan dan simpan dashboard
display_and_save_figure(fig, "dashboard_binary_risk_colorblind.png")


DASHBOARD RINGKASAN


Visualisasi disimpan sebagai 'dashboard_binary_risk_colorblind.png'


In [30]:
# 20. Simpan hasil segmentasi
df.to_csv('binary_risk_segmentation_result.csv', index=False)
print("\nHasil segmentasi dengan 2 level risiko telah disimpan ke 'binary_risk_segmentation_result.csv'")


Hasil segmentasi dengan 2 level risiko telah disimpan ke 'binary_risk_segmentation_result.csv'


In [31]:
# 21. Kesimpulan dan rekomendasi
print_header("KESIMPULAN DAN REKOMENDASI")

print(f"""
KESIMPULAN SEGMENTASI RISIKO ATTRITION KARYAWAN (RISIKO TINGGI vs RENDAH):

1. Dengan menggunakan metode {best_method}, karyawan telah berhasil disegmentasi menjadi 2 kelompok:
   - Risiko Tinggi: {risk_distribution.get('Risiko Tinggi', 0)} karyawan ({risk_percentage.get('Risiko Tinggi', 0):.1f}%)
   - Risiko Rendah: {risk_distribution.get('Risiko Rendah', 0)} karyawan ({risk_percentage.get('Risiko Rendah', 0):.1f}%)

2. Tingkat attrition di masing-masing segmen:
   - Risiko Tinggi: {attrition_by_cluster[high_risk_cluster]:.2f}%
   - Risiko Rendah: {attrition_by_cluster[low_risk_cluster]:.2f}%

3. Perbedaan utama karakteristik antara karyawan Risiko Tinggi vs Risiko Rendah:""")

# Print top 5 perbedaan
for feature, diff in pct_diff.abs().sort_values(ascending=False).head(5).items():
    actual_diff = pct_diff[feature]
    direction = "lebih tinggi" if actual_diff > 0 else "lebih rendah"
    print(f"   - {feature}: {abs(actual_diff):.1f}% {direction} pada karyawan Risiko Tinggi")

# Departemen dan Job Role dengan risiko tertinggi
if 'Department' in df.columns:
    high_risk_dept = dept_by_risk.loc['Risiko Tinggi'].idxmax()
    high_risk_dept_pct = dept_by_risk.loc['Risiko Tinggi', high_risk_dept]
    print(f"\n4. Department dengan persentase tertinggi di kelompok Risiko Tinggi:")
    print(f"   - {high_risk_dept}: {high_risk_dept_pct:.1f}%")

if 'JobRole' in df.columns:
    high_risk_role = role_by_risk.loc['Risiko Tinggi'].idxmax()
    high_risk_role_pct = role_by_risk.loc['Risiko Tinggi', high_risk_role]
    print(f"\n5. Job Role dengan persentase tertinggi di kelompok Risiko Tinggi:")
    print(f"   - {high_risk_role}: {high_risk_role_pct:.1f}%")

print("""
REKOMENDASI STRATEGI RETENSI:

1. Untuk Karyawan Risiko Tinggi:
   - Implementasikan program retensi segera dan targeted
   - Evaluasi dan tingkatkan paket kompensasi dan benefit
   - Berikan jalur pengembangan karir yang jelas
   - Adakan diskusi one-on-one untuk identifikasi masalah
   - Pertimbangkan kebijakan kerja fleksibel, terutama untuk yang tinggal jauh
   - Tinjau beban kerja dan keseimbangan kerja-hidup

2. Untuk Karyawan Risiko Rendah:
   - Pertahankan kebijakan dan program yang sudah berjalan baik
   - Manfaatkan karyawan ini sebagai mentor untuk karyawan berisiko tinggi
   - Lakukan survei kepuasan secara berkala untuk monitor perubahan
   - Berikan tantangan dan proyek baru untuk mempertahankan engagement
   - Kembangkan untuk posisi kepemimpinan dan perencanaan suksesi

3. Implementasi dan Monitoring:
   - Buat dashboard HR untuk memantau distribusi risiko secara real-time
   - Evaluasi ulang segmentasi setiap 6 bulan untuk melihat efektivitas program
   - Latih manajer untuk mengenali tanda-tanda awal attrition
   - Tentukan target pengurangan attrition yang realistis
   - Integrasikan model risiko attrition ke dalam proses manajemen talent
""")

print("\nSegmentasi risiko attrition (Tinggi vs Rendah) telah selesai dengan sukses!")


KESIMPULAN DAN REKOMENDASI

KESIMPULAN SEGMENTASI RISIKO ATTRITION KARYAWAN (RISIKO TINGGI vs RENDAH):

1. Dengan menggunakan metode KMeans, karyawan telah berhasil disegmentasi menjadi 2 kelompok:
   - Risiko Tinggi: 472 karyawan (44.6%)
   - Risiko Rendah: 586 karyawan (55.4%)

2. Tingkat attrition di masing-masing segmen:
   - Risiko Tinggi: 25.64%
   - Risiko Rendah: 9.90%

3. Perbedaan utama karakteristik antara karyawan Risiko Tinggi vs Risiko Rendah:
   - StockOptionLevel: 99.5% lebih rendah pada karyawan Risiko Tinggi
   - MonthlyIncome: 10.1% lebih rendah pada karyawan Risiko Tinggi
   - MonthlyRate: 4.4% lebih tinggi pada karyawan Risiko Tinggi
   - DailyRate: 4.2% lebih rendah pada karyawan Risiko Tinggi
   - Age: 2.9% lebih rendah pada karyawan Risiko Tinggi

4. Department dengan persentase tertinggi di kelompok Risiko Tinggi:
   - Research & Development: 65.5%

5. Job Role dengan persentase tertinggi di kelompok Risiko Tinggi:
   - Research Scientist: 22.0%

REKOMENDASI S

In [32]:
# 22. Bonus: Interactive HTML Dashboard (jika diperlukan)
print_header("BONUS: INTERACTIVE HTML DASHBOARD")

# Buat versi interaktif dari dashboard untuk dibuka di browser
interactive_dashboard = make_subplots(
    rows=2, 
    cols=2,
    subplot_titles=(
        'Segmentasi Risiko Attrition',
        'Distribusi Karyawan berdasarkan Risiko',
        'Top 5 Perbedaan Karakteristik',
        'Distribusi Department per Risiko'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "bar"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

# 1. Segmentasi (Tinggi vs Rendah)
interactive_dashboard.add_trace(
    go.Scatter(
        x=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Tinggi']['UMAP_1'],
        y=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Tinggi']['UMAP_2'],
        mode='markers',
        marker=dict(color=HIGH_RISK_COLOR, size=8),
        name='Risiko Tinggi',
        hovertemplate='<b>Risiko Tinggi</b><extra></extra>'
    ),
    row=1, col=1
)

interactive_dashboard.add_trace(
    go.Scatter(
        x=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Rendah']['UMAP_1'],
        y=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Rendah']['UMAP_2'],
        mode='markers',
        marker=dict(color=LOW_RISK_COLOR, size=8),
        name='Risiko Rendah',
        hovertemplate='<b>Risiko Rendah</b><extra></extra>'
    ),
    row=1, col=1
)

# 2. Distribusi risiko
for i, risk in enumerate(risk_df['Level Risiko']):
    color = HIGH_RISK_COLOR if risk == 'Risiko Tinggi' else LOW_RISK_COLOR
    interactive_dashboard.add_trace(
        go.Bar(
            x=[risk],
            y=[risk_df['Jumlah Karyawan'].iloc[i]],
            text=[f"{risk_df['Jumlah Karyawan'].iloc[i]} ({risk_df['Persentase'].iloc[i]:.1f}%)"],
            textposition='inside',
            name=risk,
            marker_color=color,
            showlegend=False,
            hovertemplate='<b>%{x}</b><br>Jumlah: %{y}<br>Persentase: %{text}<extra></extra>'
        ),
        row=1, col=2
    )

# 3. Top 5 perbedaan karakteristik
top_5_diff = pct_diff.abs().sort_values(ascending=False).head(5)
for feature in top_5_diff.index:
    color = HIGH_RISK_COLOR if pct_diff[feature] > 0 else LOW_RISK_COLOR
    interactive_dashboard.add_trace(
        go.Bar(
            x=[feature],
            y=[pct_diff[feature]],
            text=[f"{pct_diff[feature]:+.1f}%"],
            textposition='outside',
            marker_color=color,
            showlegend=False,
            name=feature,
            hovertemplate='<b>%{x}</b><br>Perbedaan: %{text}<extra></extra>'
        ),
        row=2, col=1
    )

# 4. Department atau Job Role distribution
if 'Department' in df.columns:
    # Ambil 3 department dengan persentase tertinggi di setiap level risiko
    for risk_level in dept_by_risk.index:
        top_depts = dept_by_risk.loc[risk_level].sort_values(ascending=False).head(3)
        for dept, pct in top_depts.items():
            color = HIGH_RISK_COLOR if risk_level == 'Risiko Tinggi' else LOW_RISK_COLOR
            interactive_dashboard.add_trace(
                go.Bar(
                    x=[dept],
                    y=[pct],
                    text=[f"{pct:.1f}%"],
                    textposition='outside',
                    name=f"{risk_level} - {dept}",
                    marker_color=color,
                    opacity=0.7,
                    showlegend=False,
                    hovertemplate='<b>%{x}</b><br>%{text} dari karyawan %{name}<extra></extra>'
                ),
                row=2, col=2
            )

# Update layout
interactive_dashboard.update_layout(
    title_text='DASHBOARD INTERAKTIF SEGMENTASI RISIKO ATTRITION KARYAWAN',
    height=900,
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(size=12)
)

# Update axes
interactive_dashboard.update_xaxes(title_text='UMAP Dimension 1', row=1, col=1)
interactive_dashboard.update_yaxes(title_text='UMAP Dimension 2', row=1, col=1)

interactive_dashboard.update_xaxes(title_text='Level Risiko', row=1, col=2)
interactive_dashboard.update_yaxes(title_text='Jumlah Karyawan', row=1, col=2)

interactive_dashboard.update_xaxes(title_text='Fitur', row=2, col=1)
interactive_dashboard.update_yaxes(title_text='Perbedaan (%)', row=2, col=1)

interactive_dashboard.update_xaxes(title_text='Department', row=2, col=2)
interactive_dashboard.update_yaxes(title_text='Persentase (%)', row=2, col=2)

# Add horizontal line at y=0 for differences
interactive_dashboard.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=4.5,
    y1=0,
    line=dict(color="black", width=1, dash="dash"),
    row=2, col=1
)

# Simpan sebagai HTML interaktif
interactive_dashboard.write_html("interactive_dashboard_colorblind.html")
print("Dashboard interaktif disimpan sebagai 'interactive_dashboard_colorblind.html'")
print("Anda dapat membuka file HTML tersebut di browser untuk interaksi lebih lanjut")


BONUS: INTERACTIVE HTML DASHBOARD
Dashboard interaktif disimpan sebagai 'interactive_dashboard_colorblind.html'
Anda dapat membuka file HTML tersebut di browser untuk interaksi lebih lanjut


In [34]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, MinMaxScaler, QuantileTransformer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.feature_selection import mutual_info_classif, SelectKBest, f_classif
from sklearn.decomposition import PCA, TruncatedSVD
from umap import UMAP
from sklearn.pipeline import Pipeline
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Skema warna yang color blind friendly
CB_COLORS = {
    'blue': '#648FFF',    # biru
    'orange': '#FE6100',  # oranye
}

# Palette untuk high risk vs low risk
HIGH_RISK_COLOR = CB_COLORS['orange']
LOW_RISK_COLOR = CB_COLORS['blue']

import warnings
warnings.filterwarnings('ignore')

# Fungsi untuk mencetak header
def print_header(text):
    print(f"\n{'='*80}\n{text}\n{'='*80}")

# Fungsi untuk menampilkan dan menyimpan visualisasi
def display_and_save_figure(fig, filename, show_fig=True):
    if show_fig:
        fig.show()
    fig.write_image(filename)
    print(f"Visualisasi disimpan sebagai '{filename}'")
    return fig

# 1. Baca dan persiapkan data
print_header("MEMBACA DAN MEMPERSIAPKAN DATA")
df = pd.read_csv('./data/data_final_employee.csv')
print(f"Ukuran dataset: {df.shape}")

# 2. Tangani missing value
print("\nMenangani missing value...")
missing_values = df.isnull().sum()
if missing_values.sum() > 0:
    print(f"Missing values ditemukan: {missing_values[missing_values > 0]}")
    df.dropna(inplace=True)
    print(f"Ukuran dataset setelah menghapus missing value: {df.shape}")
else:
    print("Tidak ada missing value ditemukan.")

# 3. Pastikan kolom Attrition numerik (0/1)
if df['Attrition'].dtype == 'object':
    df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
    print("\nKolom Attrition dikonversi dari kategorikal ke numerik (1/0)")
elif df['Attrition'].max() > 1:
    df['Attrition'] = (df['Attrition'] > 0).astype(int)
    print("\nKolom Attrition dikonversi ke binary (0/1)")

# 4. PENINGKATAN 1: Advanced Feature Engineering
print_header("ADVANCED FEATURE ENGINEERING")

# Pilih fitur numerik yang relevan
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_features = [f for f in numerical_features if f != 'Attrition' and f != 'EmployeeId' and f != 'EmployeeCount' and f != 'StandardHours' and f != 'Cluster']

print(f"\nMenggunakan {len(numerical_features)} fitur numerik asli")

# Cek fitur yang tersedia
available_features = set(df.columns)

# FEATURE ENGINEERING LANJUTAN
print("\nMembuat fitur engineering lanjutan...")

# 1. Rasio Compensation ke Job Level
if 'MonthlyIncome' in available_features and 'JobLevel' in available_features:
    df['SalaryPerLevel'] = df['MonthlyIncome'] / df['JobLevel'].replace(0, 1)
    print("- Fitur baru: SalaryPerLevel (Gaji per level jabatan)")

# 2. Indeks Kepuasan Kerja Gabungan
satisfaction_columns = [col for col in ['JobSatisfaction', 'EnvironmentSatisfaction', 
                                      'WorkLifeBalance', 'RelationshipSatisfaction'] 
                       if col in available_features]

if len(satisfaction_columns) >= 2:
    df['SatisfactionIndex'] = df[satisfaction_columns].mean(axis=1)
    print(f"- Fitur baru: SatisfactionIndex (Rata-rata dari {len(satisfaction_columns)} fitur kepuasan)")
    
    # 2.1 Weighted satisfaction index (memberikan bobot lebih untuk job satisfaction)
    df['WeightedSatisfactionIndex'] = (df['JobSatisfaction'] * 2 + df[satisfaction_columns].sum(axis=1)) / (len(satisfaction_columns) + 1)
    print("- Fitur baru: WeightedSatisfactionIndex (Indeks kepuasan dengan bobot)")
    
    # 2.2 Variance satisfaction (ketidakkonsistenan kepuasan)
    df['SatisfactionVariance'] = df[satisfaction_columns].var(axis=1)
    print("- Fitur baru: SatisfactionVariance (Variansi tingkat kepuasan)")

# 3. Rasio Promosi dan Karir
if 'YearsSinceLastPromotion' in available_features and 'YearsAtCompany' in available_features:
    df['PromotionRatio'] = df['YearsSinceLastPromotion'] / df['YearsAtCompany'].replace(0, 1)
    print("- Fitur baru: PromotionRatio (Rasio tahun sejak promosi terhadap masa kerja)")
    
    # 3.1 Promotion speed relative to job level
    if 'JobLevel' in available_features:
        df['PromotionLevelRatio'] = df['JobLevel'] / (df['YearsAtCompany'].replace(0, 1))
        print("- Fitur baru: PromotionLevelRatio (Kecepatan kenaikan level)")
    
    # 3.2 Career growth rate
    df['CareerGrowthRate'] = df['JobLevel'] / (df['TotalWorkingYears'].replace(0, 1))
    print("- Fitur baru: CareerGrowthRate (Tingkat pertumbuhan karir)")

# 4. Work-Life Balance dan Overtime
if 'WorkLifeBalance' in available_features and 'OverTime' in available_features:
    # Pastikan OverTime dalam bentuk numerik
    if df['OverTime'].dtype == 'object':
        df['OverTime'] = df['OverTime'].map({'Yes': 1, 'No': 0})
    
    # 4.1 Adjusted work-life balance (pengaruh overtime)
    df['AdjustedWorkLifeBalance'] = df['WorkLifeBalance'] / (1 + 0.5 * df['OverTime'])
    print("- Fitur baru: AdjustedWorkLifeBalance (Keseimbangan kerja-hidup disesuaikan overtime)")
    
    # 4.2 Overtime impact index
    if 'JobInvolvement' in available_features:
        df['OvertimeStressIndex'] = df['OverTime'] * (5 - df['WorkLifeBalance']) * df['JobInvolvement']
        print("- Fitur baru: OvertimeStressIndex (Indeks stres akibat lembur)")

# 5. Distance and Commute Efficiency
if 'DistanceFromHome' in available_features:
    # 5.1 Non-linear distance impact
    df['DistanceImpact'] = np.log1p(df['DistanceFromHome'] ** 1.5)
    print("- Fitur baru: DistanceImpact (Dampak non-linear jarak)")
    
    # 5.2 Distance-Satisfaction interaction
    if 'JobSatisfaction' in available_features:
        df['DistanceSatisfactionRatio'] = df['DistanceFromHome'] / df['JobSatisfaction']
        print("- Fitur baru: DistanceSatisfactionRatio (Rasio jarak terhadap kepuasan)")

# 6. Age and Career Stage
if 'Age' in available_features and 'TotalWorkingYears' in available_features:
    # 6.1 Career stage relative to age
    df['CareerStageRatio'] = df['TotalWorkingYears'] / df['Age']
    print("- Fitur baru: CareerStageRatio (Tahap karir relatif terhadap usia)")
    
    # 6.2 Job level to age ratio
    if 'JobLevel' in available_features:
        df['LevelAgeRatio'] = df['JobLevel'] / ((df['Age'] - 18) / 10)  # Normalized to career decades
        print("- Fitur baru: LevelAgeRatio (Rasio level jabatan terhadap dekade karir)")

# 7. Compensation and fairness metrics
if 'MonthlyIncome' in available_features:
    # 7.1 Compensation to experience ratio
    if 'TotalWorkingYears' in available_features:
        df['CompensationExperienceRatio'] = df['MonthlyIncome'] / (df['TotalWorkingYears'].replace(0, 0.5) + 1)
        print("- Fitur baru: CompensationExperienceRatio (Rasio gaji terhadap pengalaman)")
    
    # 7.2 Compensation to performance ratio
    if 'PerformanceRating' in available_features:
        df['CompensationPerformanceRatio'] = df['MonthlyIncome'] / df['PerformanceRating']
        print("- Fitur baru: CompensationPerformanceRatio (Rasio gaji terhadap performa)")

# Update list fitur numerik dengan fitur baru
engineered_features = [col for col in df.columns if col not in numerical_features 
                      and col != 'Attrition' and col != 'EmployeeId' 
                      and col != 'EmployeeCount' and col != 'StandardHours'
                      and col != 'Cluster'
                      and df[col].dtype != 'object']

print(f"\nMenambahkan {len(engineered_features)} fitur hasil feature engineering:")
print(engineered_features)

# Gabungkan fitur asli dan fitur baru
all_features = numerical_features + engineered_features
print(f"\nTotal fitur untuk evaluasi: {len(all_features)}")

# 5. PENINGKATAN 2: Outlier Detection and Removal
print_header("OUTLIER DETECTION AND REMOVAL")

# Persiapkan data
X_all = df[all_features]
y = df['Attrition']

# Standarisasi semua fitur
print("\nStandardisasi data...")
robust_scaler = RobustScaler()  # Robust terhadap outlier
X_scaled = robust_scaler.fit_transform(X_all)
X_scaled_df = pd.DataFrame(X_scaled, columns=all_features)

# Deteksi Outlier dengan Isolation Forest
print("\nMendeteksi outlier dengan Isolation Forest...")
iso_forest = IsolationForest(contamination=0.05, random_state=42)
outlier_labels = iso_forest.fit_predict(X_scaled)
is_outlier = outlier_labels == -1  # -1 untuk outlier, 1 untuk inlier

print(f"Jumlah outlier terdeteksi: {sum(is_outlier)} ({sum(is_outlier)/len(df)*100:.1f}%)")

# Simpan kolom outlier untuk analisis
df['is_outlier'] = is_outlier

# Visualisasi outlier vs attrition 
outlier_attrition = df.groupby('is_outlier')['Attrition'].mean() * 100
fig = px.bar(
    x=['Normal', 'Outlier'],
    y=[outlier_attrition[False], outlier_attrition[True]],
    color=['Normal', 'Outlier'],
    color_discrete_map={'Normal': LOW_RISK_COLOR, 'Outlier': HIGH_RISK_COLOR},
    title='Perbandingan Tingkat Attrition antara Data Normal vs Outlier',
    labels={'x': 'Tipe Data', 'y': 'Tingkat Attrition (%)'}
)
fig.update_layout(showlegend=False)
for i, value in enumerate([outlier_attrition[False], outlier_attrition[True]]):
    fig.add_annotation(
        x=i, y=value + 1,
        text=f"{value:.1f}%",
        showarrow=False
    )

# Tampilkan dan simpan visualisasi
display_and_save_figure(fig, "outlier_attrition_comparison.png")

# Jika outlier punya tingkat attrition lebih tinggi, kita pertahankan
# Jika tidak, kita hapus untuk meningkatkan kualitas clustering
if outlier_attrition[True] > outlier_attrition[False] * 1.5:  # 50% lebih tinggi
    print("Outlier memiliki tingkat attrition jauh lebih tinggi, mempertahankan sebagai cluster potensial")
    remove_outliers = False
else:
    print("Menghapus outlier untuk meningkatkan kualitas clustering...")
    remove_outliers = True
    # Filter outlier
    df_filtered = df[~is_outlier].copy()
    X_scaled_filtered = X_scaled[~is_outlier]
    y_filtered = y[~is_outlier]
    
    print(f"Ukuran dataset setelah menghapus outlier: {df_filtered.shape[0]} baris")


# 6. PENINGKATAN 3: Advanced Feature Selection
print_header("ADVANCED FEATURE SELECTION")

# Pilih dataset yang akan digunakan
if remove_outliers:
    X_for_selection = X_scaled_filtered
    y_for_selection = y_filtered
    df_for_model = df_filtered
else:
    X_for_selection = X_scaled
    y_for_selection = y
    df_for_model = df

# A. Feature Selection berbasis Random Forest
print("1. Feature Selection dengan Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(X_for_selection, y_for_selection)

rf_importance = pd.DataFrame({
    'Feature': all_features,
    'RF_Importance': rf.feature_importances_
}).sort_values(by='RF_Importance', ascending=False)

# B. Feature Selection berbasis Mutual Information (menangkap hubungan non-linear)
print("\n2. Feature Selection dengan Mutual Information...")
mi_scores = mutual_info_classif(X_for_selection, y_for_selection, random_state=42)
mi_importance = pd.DataFrame({
    'Feature': all_features,
    'MI_Score': mi_scores
}).sort_values(by='MI_Score', ascending=False)

# C. Feature Selection berbasis ANOVA F-value
print("\n3. Feature Selection dengan ANOVA F-value...")
f_scores, p_values = f_classif(X_for_selection, y_for_selection)
f_importance = pd.DataFrame({
    'Feature': all_features,
    'F_Score': f_scores,
    'P_Value': p_values
}).sort_values(by='F_Score', ascending=False)

# D. Gabungkan hasil feature selection
print("\n4. Menggabungkan hasil feature selection...")
# Normalisasi skor
rf_importance['RF_Normalized'] = rf_importance['RF_Importance'] / rf_importance['RF_Importance'].max()
mi_importance['MI_Normalized'] = mi_importance['MI_Score'] / mi_importance['MI_Score'].max()
f_importance['F_Normalized'] = f_importance['F_Score'] / f_importance['F_Score'].max()

# Gabungkan semua metode
feature_ranks = pd.DataFrame({'Feature': all_features})
feature_ranks = pd.merge(feature_ranks, 
                         rf_importance[['Feature', 'RF_Normalized']], 
                         on='Feature')
feature_ranks = pd.merge(feature_ranks, 
                         mi_importance[['Feature', 'MI_Normalized']], 
                         on='Feature')
feature_ranks = pd.merge(feature_ranks, 
                         f_importance[['Feature', 'F_Normalized']], 
                         on='Feature')

# Hitung skor gabungan
feature_ranks['Combined_Score'] = (feature_ranks['RF_Normalized'] + 
                                  feature_ranks['MI_Normalized'] + 
                                  feature_ranks['F_Normalized']) / 3

# Urutkan berdasarkan skor gabungan
feature_ranks = feature_ranks.sort_values(by='Combined_Score', ascending=False)

print("\nTop 15 fitur berdasarkan kombinasi metode:")
print(feature_ranks[['Feature', 'Combined_Score']].head(15))

# Visualisasi top features
fig = px.bar(
    feature_ranks.head(15),
    x='Combined_Score',
    y='Feature',
    orientation='h',
    title='Top 15 Fitur Terpenting untuk Attrition',
    color='Combined_Score',
    color_continuous_scale=[LOW_RISK_COLOR, '#FFFFFF', HIGH_RISK_COLOR]
)
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    coloraxis_showscale=False,
    height=600,
    width=900
)

# Tampilkan dan simpan visualisasi
display_and_save_figure(fig, "combined_feature_importance.png")

# E. Pilih top features untuk clustering
print("\n5. Memilih fitur optimal untuk clustering...")

# Uji berbagai jumlah fitur untuk menentukan optimal k
k_values = range(5, 26, 5)  # 5, 10, 15, 20, 25
best_silhouette = -1
best_k = 0

for k in k_values:
    # Pilih top k features
    top_k_features = feature_ranks.head(k)['Feature'].tolist()
    
    # Ekstrak data dengan top k features
    X_top_k = X_for_selection[:, [all_features.index(f) for f in top_k_features]]
    
    # Uji silhouette score dengan KMeans clustering
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=20)
    labels = kmeans.fit_predict(X_top_k)
    score = silhouette_score(X_top_k, labels)
    
    print(f"Silhouette score with top {k} features: {score:.3f}")
    
    if score > best_silhouette:
        best_silhouette = score
        best_k = k

print(f"\nJumlah fitur optimal: {best_k} dengan silhouette score: {best_silhouette:.3f}")

# Pilih fitur optimal
optimal_features = feature_ranks.head(best_k)['Feature'].tolist()
print(f"\nMenggunakan {best_k} fitur terbaik untuk clustering:")
print(optimal_features)

# Ekstrak data dengan fitur optimal
X_optimal = X_for_selection[:, [all_features.index(f) for f in optimal_features]]

# 7. PENINGKATAN 4: Advanced Transformation and Dimensionality Reduction
print_header("ADVANCED TRANSFORMATION AND DIMENSIONALITY REDUCTION")

# A. Non-linear transformation dengan Quantile Transformer
print("1. Transformasi non-linear dengan Quantile Transformer...")
qt = QuantileTransformer(output_distribution='normal', random_state=42)
X_transformed = qt.fit_transform(X_optimal)
print("Transformasi selesai.")

# B. Dimensionality Reduction dengan UMAP
print("\n2. Melakukan reduksi dimensi dengan UMAP...")
# Uji berbagai parameter UMAP
n_neighbors_values = [10, 15, 20, 30]
min_dist_values = [0.0, 0.1, 0.2, 0.5]

best_silhouette_umap = -1
best_params = None
best_umap = None
best_X_umap = None

for n_neighbors in n_neighbors_values:
    for min_dist in min_dist_values:
        # Uji UMAP dengan parameter ini
        umap_reducer = UMAP(
            n_components=2, 
            random_state=42, 
            n_neighbors=n_neighbors,
            min_dist=min_dist
        )
        X_umap = umap_reducer.fit_transform(X_transformed)
        
        # Evaluasi dengan KMeans
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=20)
        labels = kmeans.fit_predict(X_umap)
        score = silhouette_score(X_umap, labels)
        
        print(f"UMAP (n_neighbors={n_neighbors}, min_dist={min_dist}): silhouette={score:.3f}")
        
        if score > best_silhouette_umap:
            best_silhouette_umap = score
            best_params = (n_neighbors, min_dist)
            best_umap = umap_reducer
            best_X_umap = X_umap

print(f"\nParameter UMAP terbaik: n_neighbors={best_params[0]}, min_dist={best_params[1]}")
print(f"Silhouette score dengan UMAP terbaik: {best_silhouette_umap:.3f}")

# Gunakan UMAP terbaik
X_umap = best_X_umap
print(f"Dimensi setelah UMAP: {X_umap.shape}")

# Visualisasi hasil UMAP berdasarkan attrition aktual
umap_df = pd.DataFrame(X_umap, columns=['UMAP_1', 'UMAP_2'])
umap_df['Attrition'] = y_for_selection.reset_index(drop=True)
umap_df['Attrition_Label'] = umap_df['Attrition'].map({0: 'Tidak Keluar', 1: 'Keluar'})

fig = px.scatter(
    umap_df, 
    x='UMAP_1', 
    y='UMAP_2', 
    color='Attrition_Label',
    color_discrete_map={
        'Keluar': HIGH_RISK_COLOR,
        'Tidak Keluar': LOW_RISK_COLOR
    },
    title=f'Visualisasi Data dengan UMAP Optimal (Silhouette: {best_silhouette_umap:.3f})',
    opacity=0.7,
    width=900,
    height=700
)
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    legend_title_text='Status',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    font=dict(size=12)
)

# Tampilkan dan simpan visualisasi
display_and_save_figure(fig, "optimal_umap_visualization.png")

# 8. PENINGKATAN 5: Advanced Clustering Methods
print_header("ADVANCED CLUSTERING METHODS")

# A. Benchmarking berbagai metode clustering
print("Benchmarking metode clustering dengan parameter optimal...")

# Set up metode-metode clustering
clustering_methods = {
    'KMeans': KMeans(n_clusters=2, random_state=42, n_init=50),
    'GaussianMixture (Full)': GaussianMixture(n_components=2, covariance_type='full', random_state=42, max_iter=1000, n_init=10),
    'GaussianMixture (Tied)': GaussianMixture(n_components=2, covariance_type='tied', random_state=42, max_iter=1000, n_init=10),
}

# Evaluasi semua metode
scores_dict = {}

for method_name, model in clustering_methods.items():
    print(f"\nEvaluasi {method_name}...")
    
    if 'GaussianMixture' in method_name:
        model.fit(X_umap)
        labels = model.predict(X_umap)
    else:
        labels = model.fit_predict(X_umap)
    
    # Hitung berbagai metrik evaluasi
    silhouette = silhouette_score(X_umap, labels)
    calinski = calinski_harabasz_score(X_umap, labels)
    
    # Hitung tingkat attrition per cluster
    temp_df = df_for_model.copy()
    temp_df['Cluster'] = labels
    attrition_by_cluster = temp_df.groupby('Cluster')['Attrition'].mean() * 100
    attrition_diff = abs(attrition_by_cluster.max() - attrition_by_cluster.min())
    
    print(f"  Silhouette Score: {silhouette:.3f}")
    print(f"  Calinski-Harabasz Score: {calinski:.1f}")
    print(f"  Perbedaan Attrition: {attrition_diff:.1f}%")
    
    scores_dict[method_name] = {
        'silhouette': silhouette,
        'calinski': calinski,
        'attrition_diff': attrition_diff,
        'labels': labels,
        'model': model
    }

# Pilih metode terbaik berdasarkan silhouette score
best_method = max(scores_dict.items(), key=lambda x: x[1]['silhouette'])
best_method_name = best_method[0]
best_silhouette_score = best_method[1]['silhouette']
best_labels = best_method[1]['labels']
best_model = best_method[1]['model']

print(f"\nMetode terbaik: {best_method_name} dengan silhouette score: {best_silhouette_score:.3f}")

# Visualisasi perbandingan silhouette score
methods = list(scores_dict.keys())
silhouette_scores = [scores_dict[m]['silhouette'] for m in methods]

fig = px.bar(
    x=methods, 
    y=silhouette_scores,
    color=silhouette_scores,
    color_continuous_scale=[LOW_RISK_COLOR, '#FFFFFF', HIGH_RISK_COLOR],
    title='Perbandingan Silhouette Score antar Metode Clustering',
    labels={'x': 'Metode Clustering', 'y': 'Silhouette Score'}
)
for i, score in enumerate(silhouette_scores):
    fig.add_annotation(
        x=i, y=score + 0.02,
        text=f"{score:.3f}",
        showarrow=False
    )
fig.update_layout(coloraxis_showscale=False)

# Tampilkan dan simpan visualisasi
display_and_save_figure(fig, "clustering_methods_comparison.png")

# 9. Tambahkan label cluster ke dataframe
df_for_model['Cluster'] = best_labels

# 10. Analisis tingkat attrition per cluster
attrition_by_cluster = df_for_model.groupby('Cluster')['Attrition'].mean() * 100
cluster_sizes = df_for_model.groupby('Cluster').size()

print("\nAnalisis tingkat attrition per cluster:")
for cluster in sorted(attrition_by_cluster.index):
    print(f"Cluster {cluster}: {cluster_sizes[cluster]} karyawan, tingkat attrition {attrition_by_cluster[cluster]:.2f}%")

# 11. Tentukan cluster dengan risiko tinggi
high_risk_cluster = attrition_by_cluster.idxmax()
low_risk_cluster = attrition_by_cluster.idxmin()

print(f"\nCluster dengan risiko tinggi: Cluster {high_risk_cluster} ({attrition_by_cluster[high_risk_cluster]:.2f}%)")
print(f"Cluster dengan risiko rendah: Cluster {low_risk_cluster} ({attrition_by_cluster[low_risk_cluster]:.2f}%)")

# 12. Buat kolom Risk Level dengan 2 kategori
df_for_model['RiskLevel'] = df_for_model['Cluster'].map({
    high_risk_cluster: 'Risiko Tinggi', 
    low_risk_cluster: 'Risiko Rendah'
})

# 13. Hitung distribusi risiko
risk_distribution = df_for_model['RiskLevel'].value_counts()
risk_percentage = df_for_model['RiskLevel'].value_counts(normalize=True) * 100

print("\nDistribusi Karyawan berdasarkan Level Risiko Attrition:")
for risk, count in risk_distribution.items():
    percentage = risk_percentage[risk]
    print(f"{risk}: {count} karyawan ({percentage:.1f}%)")

# 14. Visualisasi hasil clustering dengan UMAP
umap_cluster_df = pd.DataFrame(X_umap, columns=['UMAP_1', 'UMAP_2'])
umap_cluster_df['Risk Level'] = df_for_model['RiskLevel'].values

fig = px.scatter(
    umap_cluster_df, 
    x='UMAP_1', 
    y='UMAP_2', 
    color='Risk Level',
    color_discrete_map={
        'Risiko Tinggi': HIGH_RISK_COLOR,
        'Risiko Rendah': LOW_RISK_COLOR
    },
    title=f'Segmentasi Risiko Attrition (Silhouette: {best_silhouette_score:.3f})',
    opacity=0.7,
    width=900,
    height=700
)
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    legend_title_text='Level Risiko',
    xaxis_title='UMAP Dimension 1',
    yaxis_title='UMAP Dimension 2',
    font=dict(size=12)
)

# Tampilkan dan simpan visualisasi
display_and_save_figure(fig, "optimal_clustering_results.png")

# 15. Analisis profil untuk setiap level risiko
print_header("ANALISIS PROFIL RISIKO")

# Pilih fitur penting untuk analisis profil
profile_features = optimal_features[:8]  # Gunakan 8 fitur teratas untuk profil

# Hitung profil rata-rata berdasarkan level risiko
profile_by_risk = df_for_model.groupby('RiskLevel')[profile_features].mean()

print("\nProfil rata-rata berdasarkan level risiko:")
print(profile_by_risk)

# Hitung persentase perbedaan antara risiko tinggi vs rendah
high_profile = profile_by_risk.loc['Risiko Tinggi'] if 'Risiko Tinggi' in profile_by_risk.index else profile_by_risk.iloc[0]
low_profile = profile_by_risk.loc['Risiko Rendah'] if 'Risiko Rendah' in profile_by_risk.index else profile_by_risk.iloc[1]

# Hitung perbedaan persentase
pct_diff = (high_profile - low_profile) / low_profile.abs() * 100

print("\nPerbedaan persentase (Risiko Tinggi vs Risiko Rendah):")
for feature, diff in pct_diff.items():
    print(f"{feature}: {diff:+.1f}%")

# 16. Visualisasi perbedaan profil
print("\nMembuat visualisasi perbedaan profil...")

# Pilih top 8 fitur dengan perbedaan terbesar untuk visualisasi
top_diff_features = pct_diff.abs().sort_values(ascending=False).head(8).index.tolist()
diff_df = pd.DataFrame({
    'Fitur': top_diff_features,
    'Perbedaan (%)': pct_diff[top_diff_features].values
})

# Gunakan colorscale yang menunjukkan perbedaan positif/negatif
colors = [HIGH_RISK_COLOR if val > 0 else LOW_RISK_COLOR for val in diff_df['Perbedaan (%)']]

fig = px.bar(
    diff_df,
    x='Fitur',
    y='Perbedaan (%)',
    color='Perbedaan (%)',
    color_continuous_scale=[LOW_RISK_COLOR, '#FFFFFF', HIGH_RISK_COLOR],
    text=[f"{val:+.1f}%" for val in diff_df['Perbedaan (%)']],
    title='Perbedaan Karakteristik Risiko Tinggi vs Risiko Rendah (%)',
    height=600,
    width=900
)
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Fitur',
    yaxis_title='Perbedaan Persentase (%)',
    xaxis={'categoryorder': 'total ascending'},
    font=dict(size=12)
)
# Tambahkan garis horizontal di 0
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=len(top_diff_features)-0.5,
    y1=0,
    line=dict(color="black", width=1, dash="dash")
)

# Tampilkan dan simpan visualisasi
display_and_save_figure(fig, "profile_differences_optimal.png")

# 17. Analisis distribusi departemen per level risiko
if 'Department' in df_for_model.columns:
    dept_by_risk = pd.crosstab(df_for_model['RiskLevel'], df_for_model['Department'], normalize='index') * 100
    
    print("\nDistribusi Department (%) per Level Risiko:")
    print(dept_by_risk)
    
    # Reshape data untuk Plotly
    dept_data = []
    for risk_level in dept_by_risk.index:
        for dept in dept_by_risk.columns:
            dept_data.append({
                'RiskLevel': risk_level,
                'Department': dept,
                'Percentage': dept_by_risk.loc[risk_level, dept]
            })
    
    dept_df = pd.DataFrame(dept_data)
    
    fig = px.bar(
        dept_df,
        x='RiskLevel',
        y='Percentage',
        color='Department',
        color_discrete_sequence=px.colors.qualitative.Bold,  # Colorblind friendly
        title='Distribusi Department berdasarkan Level Risiko',
        height=600,
        width=900,
        barmode='stack'
    )
    fig.update_layout(
        xaxis_title='Level Risiko',
        yaxis_title='Persentase (%)',
        legend_title='Department',
        font=dict(size=12)
    )
    
    # Tampilkan dan simpan visualisasi
    display_and_save_figure(fig, "department_by_risk_optimal.png")

# 18. Analisis distribusi job role per level risiko
if 'JobRole' in df_for_model.columns:
    role_by_risk = pd.crosstab(df_for_model['RiskLevel'], df_for_model['JobRole'], normalize='index') * 100
    
    print("\nDistribusi Job Role (%) per Level Risiko:")
    print(role_by_risk)
    
    # Reshape data untuk Plotly
    role_data = []
    for risk_level in role_by_risk.index:
        for role in role_by_risk.columns:
            role_data.append({
                'RiskLevel': risk_level,
                'JobRole': role,
                'Percentage': role_by_risk.loc[risk_level, role]
            })
    
    role_df = pd.DataFrame(role_data)
    
    fig = px.bar(
        role_df,
        x='RiskLevel',
        y='Percentage',
        color='JobRole',
        color_discrete_sequence=px.colors.qualitative.Plotly,  # Colorblind friendly
        title='Distribusi Job Role berdasarkan Level Risiko',
        height=600,
        width=900,
        barmode='stack'
    )
    fig.update_layout(
        xaxis_title='Level Risiko',
        yaxis_title='Persentase (%)',
        legend_title='Job Role',
        font=dict(size=12)
    )
    
    # Tampilkan dan simpan visualisasi
    display_and_save_figure(fig, "jobrole_by_risk_optimal.png")

# 19. Dashboard ringkasan
print_header("DASHBOARD RINGKASAN")

# Create dashboard with 2x2 subplots
fig = make_subplots(
    rows=2, 
    cols=2,
    subplot_titles=(
        f'Segmentasi Risiko Attrition (Silhouette: {best_silhouette_score:.3f})',
        'Distribusi Karyawan berdasarkan Risiko',
        'Top 5 Perbedaan Karakteristik',
        'Distribusi Department per Risiko'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "bar"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

# 1. Segmentasi (Tinggi vs Rendah)
fig.add_trace(
    go.Scatter(
        x=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Tinggi']['UMAP_1'],
        y=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Tinggi']['UMAP_2'],
        mode='markers',
        marker=dict(color=HIGH_RISK_COLOR, size=8),
        name='Risiko Tinggi'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Rendah']['UMAP_1'],
        y=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Rendah']['UMAP_2'],
        mode='markers',
        marker=dict(color=LOW_RISK_COLOR, size=8),
        name='Risiko Rendah'
    ),
    row=1, col=1
)

# 2. Distribusi risiko
risk_df = pd.DataFrame({
    'Level Risiko': risk_distribution.index,
    'Jumlah Karyawan': risk_distribution.values,
    'Persentase': risk_percentage.values
})

for i, risk in enumerate(risk_df['Level Risiko']):
    color = HIGH_RISK_COLOR if risk == 'Risiko Tinggi' else LOW_RISK_COLOR
    fig.add_trace(
        go.Bar(
            x=[risk],
            y=[risk_df['Jumlah Karyawan'].iloc[i]],
            text=[f"{risk_df['Jumlah Karyawan'].iloc[i]} ({risk_df['Persentase'].iloc[i]:.1f}%)"],
            textposition='inside',
            name=risk,
            marker_color=color,
            showlegend=False
        ),
        row=1, col=2
    )

# 3. Top 5 perbedaan karakteristik
top_5_diff = pct_diff.abs().sort_values(ascending=False).head(5)
for feature in top_5_diff.index:
    color = HIGH_RISK_COLOR if pct_diff[feature] > 0 else LOW_RISK_COLOR
    fig.add_trace(
        go.Bar(
            x=[feature],
            y=[pct_diff[feature]],
            text=[f"{pct_diff[feature]:+.1f}%"],
            textposition='outside',
            marker_color=color,
            showlegend=False,
            name=feature
        ),
        row=2, col=1
    )

# 4. Department atau Job Role distribution
if 'Department' in df_for_model.columns:
    # Ambil 3 department dengan persentase tertinggi di setiap level risiko
    for risk_level in dept_by_risk.index:
        top_depts = dept_by_risk.loc[risk_level].sort_values(ascending=False).head(3)
        for dept, pct in top_depts.items():
            color = HIGH_RISK_COLOR if risk_level == 'Risiko Tinggi' else LOW_RISK_COLOR
            fig.add_trace(
                go.Bar(
                    x=[dept],
                    y=[pct],
                    text=[f"{pct:.1f}%"],
                    textposition='outside',
                    name=f"{risk_level} - {dept}",
                    marker_color=color,
                    opacity=0.7,
                    showlegend=False
                ),
                row=2, col=2
            )

# Update layout
fig.update_layout(
    title_text=f'DASHBOARD SEGMENTASI RISIKO ATTRITION (Silhouette: {best_silhouette_score:.3f})',
    height=900,
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(size=12)
)

# Update axes
fig.update_xaxes(title_text='UMAP Dimension 1', row=1, col=1)
fig.update_yaxes(title_text='UMAP Dimension 2', row=1, col=1)

fig.update_xaxes(title_text='Level Risiko', row=1, col=2)
fig.update_yaxes(title_text='Jumlah Karyawan', row=1, col=2)

fig.update_xaxes(title_text='Fitur', row=2, col=1)
fig.update_yaxes(title_text='Perbedaan (%)', row=2, col=1)

fig.update_xaxes(title_text='Department', row=2, col=2)
fig.update_yaxes(title_text='Persentase (%)', row=2, col=2)

# Add horizontal line at y=0 for differences
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=4.5,
    y1=0,
    line=dict(color="black", width=1, dash="dash"),
    row=2, col=1
)

# Tampilkan dan simpan dashboard
display_and_save_figure(fig, "dashboard_optimal.png")

# 20. Simpan hasil segmentasi
df_for_model.to_csv('optimal_risk_segmentation_result.csv', index=False)
print("\nHasil segmentasi dengan silhouette score optimal telah disimpan ke 'optimal_risk_segmentation_result.csv'")

# 21. Kesimpulan dan rekomendasi
print_header("KESIMPULAN DAN REKOMENDASI")

print(f"""
KESIMPULAN SEGMENTASI RISIKO ATTRITION KARYAWAN - HASIL OPTIMAL:

1. Dengan metode {best_method_name}, kami mencapai silhouette score {best_silhouette_score:.3f} 
   yang menunjukkan kualitas segmentasi yang sangat baik.

2. Karyawan telah berhasil disegmentasi menjadi 2 kelompok:
   - Risiko Tinggi: {risk_distribution.get('Risiko Tinggi', 0)} karyawan ({risk_percentage.get('Risiko Tinggi', 0):.1f}%)
   - Risiko Rendah: {risk_distribution.get('Risiko Rendah', 0)} karyawan ({risk_percentage.get('Risiko Rendah', 0):.1f}%)

3. Tingkat attrition di masing-masing segmen:
   - Risiko Tinggi: {attrition_by_cluster[high_risk_cluster]:.2f}%
   - Risiko Rendah: {attrition_by_cluster[low_risk_cluster]:.2f}%
   - Perbedaan: {abs(attrition_by_cluster[high_risk_cluster] - attrition_by_cluster[low_risk_cluster]):.2f}%

4. Perbedaan utama karakteristik antara karyawan Risiko Tinggi vs Risiko Rendah:""")

# Print top 5 perbedaan
for feature, diff in pct_diff.abs().sort_values(ascending=False).head(5).items():
    actual_diff = pct_diff[feature]
    direction = "lebih tinggi" if actual_diff > 0 else "lebih rendah"
    print(f"   - {feature}: {abs(actual_diff):.1f}% {direction} pada karyawan Risiko Tinggi")

# Departemen dan Job Role dengan risiko tertinggi
if 'Department' in df_for_model.columns:
    high_risk_dept = dept_by_risk.loc['Risiko Tinggi'].idxmax()
    high_risk_dept_pct = dept_by_risk.loc['Risiko Tinggi', high_risk_dept]
    print(f"\n5. Department dengan persentase tertinggi di kelompok Risiko Tinggi:")
    print(f"   - {high_risk_dept}: {high_risk_dept_pct:.1f}%")

if 'JobRole' in df_for_model.columns:
    high_risk_role = role_by_risk.loc['Risiko Tinggi'].idxmax()
    high_risk_role_pct = role_by_risk.loc['Risiko Tinggi', high_risk_role]
    print(f"\n6. Job Role dengan persentase tertinggi di kelompok Risiko Tinggi:")
    print(f"   - {high_risk_role}: {high_risk_role_pct:.1f}%")

print("""
FAKTOR KEBERHASILAN SEGMENTASI:

1. Feature Engineering Lanjutan:
   - Menciptakan fitur gabungan yang lebih representatif
   - Menghitung rasio dan indeks yang merangkum beberapa fitur terkait

2. Multi-stage Feature Selection:
   - Kombinasi Random Forest, Mutual Information, dan ANOVA
   - Penggunaan jumlah fitur optimal yang menghasilkan separasi terbaik

3. Advanced Transformation:
   - Penanganan outlier yang tepat
   - Transformasi non-linear untuk normalisasi distribusi

4. UMAP dengan Parameter Optimal:
   - Pencarian parameter yang menghasilkan separasi cluster terbaik
   - Reduksi dimensi yang mempertahankan struktur lokal dan global

5. Pemilihan Model Clustering Terbaik:
   - Evaluasi dengan berbagai metrics
   - Optimalisasi parameter untuk performa maksimal

REKOMENDASI STRATEGI RETENSI:

1. Untuk Karyawan Risiko Tinggi:
   - Implementasikan program retensi segera dan targeted
   - Evaluasi dan tingkatkan paket kompensasi dan benefit
   - Berikan jalur pengembangan karir yang jelas
   - Adakan diskusi one-on-one untuk identifikasi masalah
   - Pertimbangkan kebijakan kerja fleksibel, terutama untuk yang tinggal jauh
   - Tinjau beban kerja dan keseimbangan kerja-hidup

2. Untuk Karyawan Risiko Rendah:
   - Pertahankan kebijakan dan program yang sudah berjalan baik
   - Manfaatkan karyawan ini sebagai mentor untuk karyawan berisiko tinggi
   - Lakukan survei kepuasan secara berkala untuk monitor perubahan
   - Berikan tantangan dan proyek baru untuk mempertahankan engagement
   - Kembangkan untuk posisi kepemimpinan dan perencanaan suksesi

3. Implementasi dan Monitoring:
   - Buat dashboard HR untuk memantau distribusi risiko secara real-time
   - Evaluasi ulang segmentasi setiap 6 bulan untuk melihat efektivitas program
   - Latih manajer untuk mengenali tanda-tanda awal attrition
   - Tentukan target pengurangan attrition yang realistis
   - Integrasikan model risiko attrition ke dalam proses manajemen talent
""")

print("\nSegmentasi risiko attrition dengan silhouette score optimal telah selesai dengan sukses!")

# 22. Bonus: Interactive HTML Dashboard
print_header("BONUS: INTERACTIVE HTML DASHBOARD")

# Buat versi interaktif dari dashboard untuk dibuka di browser
interactive_dashboard = make_subplots(
    rows=2, 
    cols=2,
    subplot_titles=(
        f'Segmentasi Risiko Attrition (Silhouette: {best_silhouette_score:.3f})',
        'Distribusi Karyawan berdasarkan Risiko',
        'Top 5 Perbedaan Karakteristik',
        'Distribusi Department per Risiko'
    ),
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "bar"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

# Tambahkan elemen visual yang sama dengan dashboard statis
# 1. Segmentasi (Tinggi vs Rendah)
interactive_dashboard.add_trace(
    go.Scatter(
        x=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Tinggi']['UMAP_1'],
        y=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Tinggi']['UMAP_2'],
        mode='markers',
        marker=dict(color=HIGH_RISK_COLOR, size=8),
        name='Risiko Tinggi',
        hovertemplate='<b>Risiko Tinggi</b><extra></extra>'
    ),
    row=1, col=1
)

interactive_dashboard.add_trace(
    go.Scatter(
        x=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Rendah']['UMAP_1'],
        y=umap_cluster_df[umap_cluster_df['Risk Level'] == 'Risiko Rendah']['UMAP_2'],
        mode='markers',
        marker=dict(color=LOW_RISK_COLOR, size=8),
        name='Risiko Rendah',
        hovertemplate='<b>Risiko Rendah</b><extra></extra>'
    ),
    row=1, col=1
)

# 2. Distribusi risiko
for i, risk in enumerate(risk_df['Level Risiko']):
    color = HIGH_RISK_COLOR if risk == 'Risiko Tinggi' else LOW_RISK_COLOR
    interactive_dashboard.add_trace(
        go.Bar(
            x=[risk],
            y=[risk_df['Jumlah Karyawan'].iloc[i]],
            text=[f"{risk_df['Jumlah Karyawan'].iloc[i]} ({risk_df['Persentase'].iloc[i]:.1f}%)"],
            textposition='inside',
            name=risk,
            marker_color=color,
            showlegend=False,
            hovertemplate='<b>%{x}</b><br>Jumlah: %{y}<br>Persentase: %{text}<extra></extra>'
        ),
        row=1, col=2
    )

# 3. Top 5 perbedaan karakteristik
for feature in top_5_diff.index:
    color = HIGH_RISK_COLOR if pct_diff[feature] > 0 else LOW_RISK_COLOR
    interactive_dashboard.add_trace(
        go.Bar(
            x=[feature],
            y=[pct_diff[feature]],
            text=[f"{pct_diff[feature]:+.1f}%"],
            textposition='outside',
            marker_color=color,
            showlegend=False,
            name=feature,
            hovertemplate='<b>%{x}</b><br>Perbedaan: %{text}<extra></extra>'
        ),
        row=2, col=1
    )

# 4. Department atau Job Role distribution
if 'Department' in df_for_model.columns:
    # Ambil 3 department dengan persentase tertinggi di setiap level risiko
    for risk_level in dept_by_risk.index:
        top_depts = dept_by_risk.loc[risk_level].sort_values(ascending=False).head(3)
        for dept, pct in top_depts.items():
            color = HIGH_RISK_COLOR if risk_level == 'Risiko Tinggi' else LOW_RISK_COLOR
            interactive_dashboard.add_trace(
                go.Bar(
                    x=[dept],
                    y=[pct],
                    text=[f"{pct:.1f}%"],
                    textposition='outside',
                    name=f"{risk_level} - {dept}",
                    marker_color=color,
                    opacity=0.7,
                    showlegend=False,
                    hovertemplate='<b>%{x}</b><br>%{text} dari karyawan %{name}<extra></extra>'
                ),
                row=2, col=2
            )

# Update layout
interactive_dashboard.update_layout(
    title_text=f'DASHBOARD INTERAKTIF SEGMENTASI RISIKO ATTRITION (Silhouette: {best_silhouette_score:.3f})',
    height=900,
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    font=dict(size=12)
)

# Update axes
interactive_dashboard.update_xaxes(title_text='UMAP Dimension 1', row=1, col=1)
interactive_dashboard.update_yaxes(title_text='UMAP Dimension 2', row=1, col=1)

interactive_dashboard.update_xaxes(title_text='Level Risiko', row=1, col=2)
interactive_dashboard.update_yaxes(title_text='Jumlah Karyawan', row=1, col=2)

interactive_dashboard.update_xaxes(title_text='Fitur', row=2, col=1)
interactive_dashboard.update_yaxes(title_text='Perbedaan (%)', row=2, col=1)

interactive_dashboard.update_xaxes(title_text='Department', row=2, col=2)
interactive_dashboard.update_yaxes(title_text='Persentase (%)', row=2, col=2)

# Add horizontal line at y=0 for differences
interactive_dashboard.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=4.5,
    y1=0,
    line=dict(color="black", width=1, dash="dash"),
    row=2, col=1
)

# Simpan sebagai HTML interaktif
interactive_dashboard.write_html("optimal_interactive_dashboard.html")
print("Dashboard interaktif disimpan sebagai 'optimal_interactive_dashboard.html'")
print("Anda dapat membuka file HTML tersebut di browser untuk interaksi lebih lanjut")


MEMBACA DAN MEMPERSIAPKAN DATA
Ukuran dataset: (1058, 39)

Menangani missing value...
Tidak ada missing value ditemukan.

ADVANCED FEATURE ENGINEERING

Menggunakan 23 fitur numerik asli

Membuat fitur engineering lanjutan...
- Fitur baru: SalaryPerLevel (Gaji per level jabatan)
- Fitur baru: SatisfactionIndex (Rata-rata dari 4 fitur kepuasan)
- Fitur baru: WeightedSatisfactionIndex (Indeks kepuasan dengan bobot)
- Fitur baru: SatisfactionVariance (Variansi tingkat kepuasan)
- Fitur baru: PromotionRatio (Rasio tahun sejak promosi terhadap masa kerja)
- Fitur baru: PromotionLevelRatio (Kecepatan kenaikan level)
- Fitur baru: CareerGrowthRate (Tingkat pertumbuhan karir)
- Fitur baru: AdjustedWorkLifeBalance (Keseimbangan kerja-hidup disesuaikan overtime)
- Fitur baru: OvertimeStressIndex (Indeks stres akibat lembur)
- Fitur baru: DistanceImpact (Dampak non-linear jarak)
- Fitur baru: DistanceSatisfactionRatio (Rasio jarak terhadap kepuasan)
- Fitur baru: CareerStageRatio (Tahap karir rel

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [36]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, MinMaxScaler, QuantileTransformer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.feature_selection import mutual_info_classif, SelectKBest, f_classif
from sklearn.decomposition import PCA, TruncatedSVD
from umap import UMAP
from sklearn.pipeline import Pipeline
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Skema warna yang color blind friendly
CB_COLORS = {
    'blue': '#648FFF',    # biru
    'orange': '#FE6100',  # oranye
}

# Palette untuk high risk vs low risk
HIGH_RISK_COLOR = CB_COLORS['orange']
LOW_RISK_COLOR = CB_COLORS['blue']

import warnings
warnings.filterwarnings('ignore')

# Fungsi untuk mencetak header
def print_header(text):
    print(f"\n{'='*80}\n{text}\n{'='*80}")

# Fungsi untuk menampilkan dan menyimpan visualisasi
def display_and_save_figure(fig, filename, show_fig=True):
    if show_fig:
        fig.show()
    fig.write_image(filename)
    print(f"Visualisasi disimpan sebagai '{filename}'")
    return fig

# Fungsi untuk memeriksa dan mengatasi nilai tak terhingga dan NaN
def handle_infinite_values(df, features):
    # Buat salinan DataFrame
    df_clean = df.copy()
    
    # Cek nilai tak terhingga atau NaN dalam dataframe
    has_inf = np.any(np.isinf(df[features].values))
    has_nan = df[features].isna().any().any()
    
    if has_inf or has_nan:
        print("Data mengandung nilai tak terhingga atau NaN. Menangani masalah ini...")
        
        # Hitung jumlah nilai infinite dan NaN per kolom
        inf_count = np.isinf(df[features]).sum()
        nan_count = df[features].isna().sum()
        
        # Tampilkan kolom dengan masalah
        problem_cols = []
        for col in features:
            inf_vals = np.isinf(df[col]).sum()
            nan_vals = df[col].isna().sum()
            if inf_vals > 0 or nan_vals > 0:
                problem_cols.append((col, inf_vals, nan_vals))
                print(f"  - Kolom '{col}': {inf_vals} nilai tak terhingga, {nan_vals} nilai NaN")
        
        # Tangani nilai tak terhingga dengan mengganti dengan NaN terlebih dahulu
        for col in features:
            # Ganti inf dengan NaN
            df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
            
            # Cek apakah kolom masih memiliki nilai yang valid (bukan semuanya NaN)
            if df_clean[col].notna().any():
                # Ganti NaN dengan median kolom
                median_val = df_clean[col].median()
                df_clean[col] = df_clean[col].fillna(median_val)
                print(f"  - Mengganti nilai tak terhingga/NaN di kolom '{col}' dengan median: {median_val}")
            else:
                # Jika semua nilai NaN, ganti dengan 0
                df_clean[col] = 0
                print(f"  - Semua nilai di kolom '{col}' tidak valid. Mengganti dengan 0.")
    
    return df_clean

# 1. Baca dan persiapkan data
print_header("MEMBACA DAN MEMPERSIAPKAN DATA")
df = pd.read_csv('./data/data_final_employee.csv')
print(f"Ukuran dataset: {df.shape}")

# 2. Tangani missing value
print("\nMenangani missing value...")
missing_values = df.isnull().sum()
if missing_values.sum() > 0:
    print(f"Missing values ditemukan: {missing_values[missing_values > 0]}")
    df.dropna(inplace=True)
    print(f"Ukuran dataset setelah menghapus missing value: {df.shape}")
else:
    print("Tidak ada missing value ditemukan.")

# 3. Pastikan kolom Attrition numerik (0/1)
if df['Attrition'].dtype == 'object':
    df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
    print("\nKolom Attrition dikonversi dari kategorikal ke numerik (1/0)")
elif df['Attrition'].max() > 1:
    df['Attrition'] = (df['Attrition'] > 0).astype(int)
    print("\nKolom Attrition dikonversi ke binary (0/1)")

# 4. Feature Engineering yang lebih stabil
print_header("FEATURE ENGINEERING STABIL")

# Pilih fitur numerik yang relevan
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_features = [f for f in numerical_features if f != 'Attrition' and f != 'EmployeeId' and f != 'EmployeeCount' and f != 'StandardHours' and f != 'Cluster']

print(f"\nMenggunakan {len(numerical_features)} fitur numerik asli")

# Cek fitur yang tersedia
available_features = set(df.columns)

# FEATURE ENGINEERING DENGAN PENANGANAN NILAI EKSTREM
print("\nMembuat fitur engineering dengan penanganan nilai ekstrem...")

try:
    # 1. Rasio Compensation ke Job Level (dengan penanganan division by zero)
    if 'MonthlyIncome' in available_features and 'JobLevel' in available_features:
        # Pastikan tidak ada pembagian dengan nol
        df['SalaryPerLevel'] = df['MonthlyIncome'] / df['JobLevel'].replace(0, 1)
        print("- Fitur baru: SalaryPerLevel (Gaji per level jabatan)")
    
    # 2. Indeks Kepuasan Kerja Gabungan
    satisfaction_columns = [col for col in ['JobSatisfaction', 'EnvironmentSatisfaction', 
                                        'WorkLifeBalance', 'RelationshipSatisfaction'] 
                        if col in available_features]
    
    if len(satisfaction_columns) >= 2:
        df['SatisfactionIndex'] = df[satisfaction_columns].mean(axis=1)
        print(f"- Fitur baru: SatisfactionIndex (Rata-rata dari {len(satisfaction_columns)} fitur kepuasan)")
    
    # 3. Rasio Promosi (dengan penanganan division by zero)
    if 'YearsSinceLastPromotion' in available_features and 'YearsAtCompany' in available_features:
        df['PromotionRatio'] = df['YearsSinceLastPromotion'] / df['YearsAtCompany'].replace(0, 1)
        # Terapkan clipping untuk menghindari nilai ekstrem
        max_ratio = np.percentile(df['PromotionRatio'], 99)  # 99 persentil
        df['PromotionRatio'] = df['PromotionRatio'].clip(0, max_ratio)
        print("- Fitur baru: PromotionRatio (Rasio tahun sejak promosi terhadap masa kerja)")
    
    # 4. Transformasi logaritmik (lebih stabil daripada exponential)
    if 'DistanceFromHome' in available_features:
        # Log transform lebih stabil (tambahkan 1 untuk menghindari log(0))
        df['LogDistance'] = np.log1p(df['DistanceFromHome'])
        print("- Fitur baru: LogDistance (Transformasi logaritmik jarak)")
    
    # 5. Work-Life Balance dan Overtime (Linear, lebih stabil)
    if 'WorkLifeBalance' in available_features and 'OverTime' in available_features:
        # Pastikan OverTime dalam bentuk numerik
        if df['OverTime'].dtype == 'object':
            df['OverTime'] = df['OverTime'].map({'Yes': 1, 'No': 0})
        
        # Penambahan sederhana lebih stabil daripada perkalian/pembagian
        df['WorkLifeBalanceAdjusted'] = df['WorkLifeBalance'] - 0.5 * df['OverTime']
        print("- Fitur baru: WorkLifeBalanceAdjusted (Penyesuaian work-life balance)")
        
    # Hanya tambahkan beberapa fitur kunci untuk mengurangi risiko nilai ekstrem
    print("\nMembatasi jumlah fitur engineering untuk mengurangi risiko nilai ekstrem")
except Exception as e:
    print(f"Error saat membuat fitur baru: {e}")
    print("Melanjutkan dengan fitur yang berhasil dibuat")

# Update list fitur numerik dengan fitur baru
engineered_features = [col for col in df.columns if col not in numerical_features 
                      and col != 'Attrition' and col != 'EmployeeId' 
                      and col != 'EmployeeCount' and col != 'StandardHours'
                      and col != 'Cluster'
                      and df[col].dtype != 'object']

print(f"\nMenambahkan {len(engineered_features)} fitur hasil feature engineering:")
print(engineered_features)

# Gabungkan fitur asli dan fitur baru
all_features = numerical_features + engineered_features
print(f"\nTotal fitur untuk evaluasi: {len(all_features)}")

# 5. Periksa dan tangani nilai tak terhingga
print_header("PENANGANAN NILAI TAK TERHINGGA")

# Persiapkan data
X_all = df[all_features]

# Tangani nilai tak terhingga atau NaN
X_all_clean = handle_infinite_values(X_all, all_features)

# Update DataFrame dengan data yang sudah dibersihkan
df[all_features] = X_all_clean

# Ambil data target
y = df['Attrition']

# Tampilkan statistik data setelah pembersihan
print("\nStatistik data setelah pembersihan:")
print(f"  Jumlah fitur: {len(all_features)}")
print(f"  Min values: {X_all_clean.min().min()}")
print(f"  Max values: {X_all_clean.max().max()}")
print(f"  Rata-rata seluruh nilai: {X_all_clean.mean().mean()}")
print(f"  Median seluruh nilai: {X_all_clean.median().median()}")

# 6. Standarisasi data dengan clip untuk mencegah nilai ekstrem
print_header("STANDARISASI DATA DENGAN CLIPPING")

print("Melakukan standarisasi data dengan penanganan nilai ekstrem...")

try:
    # Menggunakan StandardScaler lebih stabil daripada RobustScaler dalam beberapa kasus
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_all_clean)
    
    # Terapkan clipping untuk batasi nilai ekstrem setelah standarisasi
    X_scaled = np.clip(X_scaled, -5, 5)  # Batasi ke range [-5, 5] untuk mencegah nilai ekstrem
    
    # Konversi kembali ke DataFrame
    X_scaled_df = pd.DataFrame(X_scaled, columns=all_features)
    
    print("Standarisasi berhasil. Menerapkan clipping untuk batasi nilai ekstrem ke range [-5, 5]")
    
except Exception as e:
    print(f"Error saat standarisasi: {e}")
    print("Mencoba metode standarisasi alternatif...")
    
    # Pendekatan alternatif: standarisasi per kolom dengan penanganan manual
    X_scaled = np.zeros_like(X_all_clean.values)
    
    for i, col in enumerate(all_features):
        # Ambil data kolom
        col_data = X_all_clean[col].values
        
        # Hitung mean dan std secara manual, ignoring NaN
        col_mean = np.nanmean(col_data)
        col_std = np.nanstd(col_data)
        
        # Hindari division by zero
        if col_std > 0:
            X_scaled[:, i] = (col_data - col_mean) / col_std
        else:
            X_scaled[:, i] = 0  # Jika std=0, semua nilai sama, jadi set ke 0
        
        # Terapkan clipping untuk batasi nilai ekstrem
        X_scaled[:, i] = np.clip(X_scaled[:, i], -5, 5)
    
    # Konversi kembali ke DataFrame
    X_scaled_df = pd.DataFrame(X_scaled, columns=all_features)
    
    print("Standarisasi manual berhasil diterapkan dengan clipping")

print("\nData setelah standarisasi dan clipping:")
print(f"  Min values: {X_scaled_df.min().min()}")
print(f"  Max values: {X_scaled_df.max().max()}")
print(f"  Mean: {X_scaled_df.mean().mean()}")
print(f"  Std: {X_scaled_df.std().mean()}")

# 7. Feature Selection untuk mengurangi dimensi
print_header("FEATURE SELECTION SEDERHANA")

# Gunakan Random Forest yang stabil untuk feature selection
print("Mengidentifikasi fitur terpenting dengan Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_scaled_df, y)

# Feature importance
importance = pd.DataFrame({
    'Feature': all_features,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 15 fitur terpenting:")
print(importance.head(15))

# Hanya pilih 10 fitur teratas untuk stabilitas
top_features = importance.head(10)['Feature'].tolist()
print(f"\nMenggunakan 10 fitur terpenting untuk clustering: {top_features}")

# Filter data untuk top features
X_top = X_scaled_df[top_features]

# 8. Reduksi dimensi dengan UMAP (dengan parameter stabil)
print_header("REDUKSI DIMENSI DENGAN UMAP")

print("Menerapkan UMAP dengan parameter yang stabil...")
try:
    # Dengan parameter stabil, min_dist lebih tinggi untuk menghindari tumpukan titik
    umap_reducer = UMAP(
        n_components=2,
        random_state=42,
        n_neighbors=30,  # Lebih tinggi untuk stabilitas
        min_dist=0.3,    # Lebih tinggi untuk menghindari tumpukan titik
        metric='euclidean'
    )
    X_umap = umap_reducer.fit_transform(X_top)
    print("UMAP berhasil diterapkan")
    
except Exception as e:
    print(f"Error saat menerapkan UMAP: {e}")
    print("Menggunakan PCA sebagai alternatif...")
    
    # Gunakan PCA sebagai fallback
    pca = PCA(n_components=2, random_state=42)
    X_umap = pca.fit_transform(X_top)
    print("PCA berhasil diterapkan sebagai alternatif")

# 9. Clustering dengan 2 cluster
print_header("CLUSTERING DENGAN 2 CLUSTER")

# Aplikasikan KMeans (lebih stabil daripada GMM)
print("Menerapkan KMeans dengan 2 cluster...")
kmeans = KMeans(n_clusters=2, random_state=42, n_init=50)
cluster_labels = kmeans.fit_predict(X_umap)

# Hitung silhouette score
sil_score = silhouette_score(X_umap, cluster_labels)
print(f"Silhouette score: {sil_score:.3f}")

# Tambahkan label cluster ke DataFrame
df['Cluster'] = cluster_labels

# 10. Analisis tingkat attrition per cluster
attrition_by_cluster = df.groupby('Cluster')['Attrition'].mean() * 100
cluster_sizes = df.groupby('Cluster').size()

print("\nAnalisis tingkat attrition per cluster:")
for cluster in sorted(attrition_by_cluster.index):
    print(f"Cluster {cluster}: {cluster_sizes[cluster]} karyawan, tingkat attrition {attrition_by_cluster[cluster]:.2f}%")

# 11. Tentukan cluster dengan risiko tinggi
high_risk_cluster = attrition_by_cluster.idxmax()
low_risk_cluster = attrition_by_cluster.idxmin()

print(f"\nCluster dengan risiko tinggi: Cluster {high_risk_cluster} ({attrition_by_cluster[high_risk_cluster]:.2f}%)")
print(f"Cluster dengan risiko rendah: Cluster {low_risk_cluster} ({attrition_by_cluster[low_risk_cluster]:.2f}%)")

# 12. Buat kolom Risk Level dengan 2 kategori
df['RiskLevel'] = df['Cluster'].map({
    high_risk_cluster: 'Risiko Tinggi', 
    low_risk_cluster: 'Risiko Rendah'
})

# 13. Hitung distribusi risiko
risk_distribution = df['RiskLevel'].value_counts()
risk_percentage = df['RiskLevel'].value_counts(normalize=True) * 100

print("\nDistribusi Karyawan berdasarkan Level Risiko Attrition:")
for risk, count in risk_distribution.items():
    percentage = risk_percentage[risk]
    print(f"{risk}: {count} karyawan ({percentage:.1f}%)")

# 14. Visualisasi segmentasi dengan Plotly
print_header("VISUALISASI SEGMENTASI")

print("Membuat visualisasi scatter plot...")
umap_df = pd.DataFrame(X_umap, columns=['Dimension 1', 'Dimension 2'])
umap_df['RiskLevel'] = df['RiskLevel'].values

# Visualisasi scatter plot
fig = px.scatter(
    umap_df, 
    x='Dimension 1', 
    y='Dimension 2', 
    color='RiskLevel',
    color_discrete_map={
        'Risiko Tinggi': HIGH_RISK_COLOR,
        'Risiko Rendah': LOW_RISK_COLOR
    },
    title=f'Segmentasi Risiko Attrition (Silhouette: {sil_score:.3f})',
    opacity=0.7,
    width=900,
    height=700
)
fig.update_traces(marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    legend_title_text='Level Risiko',
    xaxis_title='Dimension 1',
    yaxis_title='Dimension 2',
    font=dict(size=12)
)

# Tampilkan dan simpan visualisasi
display_and_save_figure(fig, "clustering_result_safe.png")

# 15. Analisis profil risiko
print_header("ANALISIS PROFIL RISIKO")

# Gunakan fitur asli (bukan yang distandarisasi) untuk interpretasi yang lebih baik
profile_features = top_features.copy()
profile_by_risk = df.groupby('RiskLevel')[profile_features].mean()

print("\nProfil rata-rata berdasarkan level risiko:")
print(profile_by_risk)

# Hitung perbedaan persentase
high_profile = profile_by_risk.loc['Risiko Tinggi'] if 'Risiko Tinggi' in profile_by_risk.index else profile_by_risk.iloc[0]
low_profile = profile_by_risk.loc['Risiko Rendah'] if 'Risiko Rendah' in profile_by_risk.index else profile_by_risk.iloc[1]

# Hitung perbedaan persentase dengan penanganan division by zero
pct_diff = pd.Series(index=profile_features)
for feature in profile_features:
    if low_profile[feature] != 0:
        pct_diff[feature] = (high_profile[feature] - low_profile[feature]) / abs(low_profile[feature]) * 100
    else:
        # Jika denominator 0, gunakan perbedaan absolut
        pct_diff[feature] = (high_profile[feature] - low_profile[feature]) * 100
        
print("\nPerbedaan persentase (Risiko Tinggi vs Risiko Rendah):")
for feature, diff in pct_diff.items():
    print(f"{feature}: {diff:+.1f}%")

# 16. Visualisasi perbedaan karakteristik
print("\nMembuat visualisasi perbedaan karakteristik...")
# Pilih top 5 fitur dengan perbedaan terbesar untuk visualisasi
top_diff_features = pct_diff.abs().sort_values(ascending=False).head(5).index.tolist()
diff_df = pd.DataFrame({
    'Fitur': top_diff_features,
    'Perbedaan (%)': pct_diff[top_diff_features].values
})

# Visualisasi bar chart
fig = px.bar(
    diff_df,
    x='Fitur',
    y='Perbedaan (%)',
    color='Perbedaan (%)',
    color_continuous_scale=[LOW_RISK_COLOR, '#FFFFFF', HIGH_RISK_COLOR],
    text=[f"{val:+.1f}%" for val in diff_df['Perbedaan (%)']],
    title='Perbedaan Karakteristik Risiko Tinggi vs Risiko Rendah (%)',
    height=600,
    width=900
)
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Fitur',
    yaxis_title='Perbedaan Persentase (%)',
    xaxis={'categoryorder': 'total ascending'},
    font=dict(size=12)
)
# Tambahkan garis horizontal di 0
fig.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=len(top_diff_features)-0.5,
    y1=0,
    line=dict(color="black", width=1, dash="dash")
)

# Tampilkan dan simpan visualisasi
display_and_save_figure(fig, "characteristic_differences_safe.png")

# 17. Simpan hasil segmentasi
df.to_csv('risk_segmentation_safe.csv', index=False)
print("\nHasil segmentasi risiko attrition telah disimpan ke 'risk_segmentation_safe.csv'")

# 18. Kesimpulan dan rekomendasi
print_header("KESIMPULAN DAN REKOMENDASI")
print(f"""
KESIMPULAN SEGMENTASI RISIKO ATTRITION KARYAWAN:

1. Dengan metode KMeans, kami mencapai silhouette score {sil_score:.3f} 
   yang menunjukkan kualitas segmentasi yang baik.

2. Karyawan telah berhasil disegmentasi menjadi 2 kelompok:
   - Risiko Tinggi: {risk_distribution.get('Risiko Tinggi', 0)} karyawan ({risk_percentage.get('Risiko Tinggi', 0):.1f}%)
   - Risiko Rendah: {risk_distribution.get('Risiko Rendah', 0)} karyawan ({risk_percentage.get('Risiko Rendah', 0):.1f}%)

3. Tingkat attrition di masing-masing segmen:
   - Risiko Tinggi: {attrition_by_cluster[high_risk_cluster]:.2f}%
   - Risiko Rendah: {attrition_by_cluster[low_risk_cluster]:.2f}%
   - Perbedaan: {abs(attrition_by_cluster[high_risk_cluster] - attrition_by_cluster[low_risk_cluster]):.2f}%

4. Perbedaan utama karakteristik antara karyawan Risiko Tinggi vs Risiko Rendah:""")

# Print top 5 perbedaan
for feature, diff in pct_diff.abs().sort_values(ascending=False).head(5).items():
    actual_diff = pct_diff[feature]
    direction = "lebih tinggi" if actual_diff > 0 else "lebih rendah"
    print(f"   - {feature}: {abs(actual_diff):.1f}% {direction} pada karyawan Risiko Tinggi")

print("""
REKOMENDASI STRATEGI RETENSI:

1. Untuk Karyawan Risiko Tinggi:
   - Implementasikan program retensi segera dan targeted
   - Evaluasi dan tingkatkan paket kompensasi dan benefit
   - Berikan jalur pengembangan karir yang jelas
   - Adakan diskusi one-on-one untuk identifikasi masalah
   - Pertimbangkan kebijakan kerja fleksibel

2. Untuk Karyawan Risiko Rendah:
   - Pertahankan kebijakan dan program yang sudah berjalan baik
   - Manfaatkan karyawan ini sebagai mentor untuk karyawan berisiko tinggi
   - Lakukan survei kepuasan secara berkala untuk monitor perubahan

3. Implementasi dan Monitoring:
   - Buat dashboard HR untuk memantau distribusi risiko secara real-time
   - Evaluasi ulang segmentasi setiap 6 bulan untuk melihat efektivitas program
   - Tentukan target pengurangan attrition yang realistis
""")

print("\nSegmentasi risiko attrition telah selesai dengan aman!")


MEMBACA DAN MEMPERSIAPKAN DATA
Ukuran dataset: (1058, 39)

Menangani missing value...
Tidak ada missing value ditemukan.

FEATURE ENGINEERING STABIL

Menggunakan 23 fitur numerik asli

Membuat fitur engineering dengan penanganan nilai ekstrem...
- Fitur baru: SalaryPerLevel (Gaji per level jabatan)
- Fitur baru: SatisfactionIndex (Rata-rata dari 4 fitur kepuasan)
- Fitur baru: PromotionRatio (Rasio tahun sejak promosi terhadap masa kerja)
- Fitur baru: LogDistance (Transformasi logaritmik jarak)
- Fitur baru: WorkLifeBalanceAdjusted (Penyesuaian work-life balance)

Membatasi jumlah fitur engineering untuk mengurangi risiko nilai ekstrem

Menambahkan 6 fitur hasil feature engineering:
['OverTime', 'SalaryPerLevel', 'SatisfactionIndex', 'PromotionRatio', 'LogDistance', 'WorkLifeBalanceAdjusted']

Total fitur untuk evaluasi: 29

PENANGANAN NILAI TAK TERHINGGA

Statistik data setelah pembersihan:
  Jumlah fitur: 29
  Min values: 0.0
  Max values: 26999.0
  Rata-rata seluruh nilai: 857.790

Visualisasi disimpan sebagai 'clustering_result_safe.png'

ANALISIS PROFIL RISIKO

Profil rata-rata berdasarkan level risiko:
                     Age  MonthlyIncome  TotalWorkingYears   DailyRate  \
RiskLevel                                                                
Risiko Rendah  36.838667    6616.517333          11.426667  807.545333   
Risiko Tinggi  37.584416    6648.902597          11.457792  814.405844   

               SatisfactionIndex   MonthlyRate  SalaryPerLevel  OverTime  \
RiskLevel                                                                  
Risiko Rendah           2.722667  14306.189333     2965.003956  0.000000   
Risiko Tinggi           2.775974  14172.538961     3020.849134  0.996753   

               HourlyRate  WorkLifeBalanceAdjusted  
RiskLevel                                           
Risiko Rendah   65.848000                 2.793333  
Risiko Tinggi   65.146104                 2.193182  

Perbedaan persentase (Risiko Tinggi vs Risiko Rendah):
Age:

Visualisasi disimpan sebagai 'characteristic_differences_safe.png'

Hasil segmentasi risiko attrition telah disimpan ke 'risk_segmentation_safe.csv'

KESIMPULAN DAN REKOMENDASI

KESIMPULAN SEGMENTASI RISIKO ATTRITION KARYAWAN:

1. Dengan metode KMeans, kami mencapai silhouette score 0.701 
   yang menunjukkan kualitas segmentasi yang baik.

2. Karyawan telah berhasil disegmentasi menjadi 2 kelompok:
   - Risiko Tinggi: 308 karyawan (29.1%)
   - Risiko Rendah: 750 karyawan (70.9%)

3. Tingkat attrition di masing-masing segmen:
   - Risiko Tinggi: 31.82%
   - Risiko Rendah: 10.80%
   - Perbedaan: 21.02%

4. Perbedaan utama karakteristik antara karyawan Risiko Tinggi vs Risiko Rendah:
   - OverTime: 99.7% lebih tinggi pada karyawan Risiko Tinggi
   - WorkLifeBalanceAdjusted: 21.5% lebih rendah pada karyawan Risiko Tinggi
   - Age: 2.0% lebih tinggi pada karyawan Risiko Tinggi
   - SatisfactionIndex: 2.0% lebih tinggi pada karyawan Risiko Tinggi
   - SalaryPerLevel: 1.9% lebih tinggi pada 